<a href="https://colab.research.google.com/github/0mehedihasan/Machine-Learning-Course/blob/main/ML_RUN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd

df = pd.read_csv('/content/drive/MyDrive/ML COURSE PAPER/DATASET/Parkinsson disease.csv')
display(df.head())

,name,MDVP:Fo(Hz),MDVP:Fhi(Hz),MDVP:Flo(Hz),MDVP:Jitter(%),MDVP:Jitter(Abs),MDVP:RAP,MDVP:PPQ,Jitter:DDP,MDVP:Shimmer,...,Shimmer:DDA,NHR,HNR,status,RPDE,DFA,spread1,spread2,D2,PPE
0,phon_R01_S01_1,119.992,157.302,74.997,0.00784,0.00007,0.00370,0.00554,0.01109,0.04374,...,0.06545,0.02211,21.033,1,0.414783,0.815285,-4.813031,0.266482,2.301442,0.284654
1,phon_R01_S01_2,122.400,148.650,113.819,0.00968,0.00008,0.00465,0.00696,0.01394,0.06134,...,0.09403,0.01929,19.085,1,0.458359,0.819521,-4.075192,0.335590,2.486855,0.368674
2,phon_R01_S01_3,116.682,131.111,111.555,0.01050,0.00009,0.00544,0.00781,0.01633,0.05233,...,0.08270,0.01309,20.651,1,0.429895,0.825288,-4.443179,0.311173,2.342259,0.332634
3,phon_R01_S01_4,116.676,137.871,111.366,0.00997,0.00009,0.00502,0.00698,0.01505,0.05492,...,0.08771,0.01353,20.644,1,0.434969,0.819235,-4.117501,0.334147,2.405554,0.368975
4,phon_R01_S01_5,116.014,141.781,110.655,0.01284,0.00011,0.00655,0.00908,0.01966,0.06425,...,0.10470,0.01767,19.649,1,0.417356,0.823484,-3.747787,0.234513,2.332180,0.410335


In [6]:
# Colab-ready end-to-end pipeline for:
# "Machine Learning Approaches for Early Detection and Severity Prediction of Parkinson’s Disease from Voice Features"
# Paste into a single Colab cell and run.
# NOTE: In Colab you have internet; the script installs required packages automatically.

# ---------- 1) Environment setup ----------
!pip install -q pandas numpy scikit-learn matplotlib xgboost lightgbm shap imbalanced-learn joblib scipy

# boruta_shap is optional; try to install but continue if it fails.
try:
    !pip install -q boruta_shap
    BORUTA_AVAILABLE = True
except Exception as e:
    print("boruta_shap install failed or unavailable; falling back to SelectKBest. Error:", e)
    BORUTA_AVAILABLE = False

# ---------- 2) Imports ----------
import os, sys, warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, mutual_info_classif, mutual_info_regression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LinearRegression, ElasticNet
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score, roc_auc_score, confusion_matrix
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBClassifier, XGBRegressor
import lightgbm as lgb
import joblib
from imblearn.over_sampling import SMOTE
import re

# Optional imports
try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False

try:
    from boruta_shap import BorutaShap
    BORUTA_AVAILABLE = True
except Exception:
    BORUTA_AVAILABLE = BORUTA_AVAILABLE  # keep earlier value

# ---------- 3) Load your CSV ----------
# Update csv_path if you uploaded to a different location in Colab
csv_path = "/content/drive/MyDrive/ML COURSE PAPER/DATASET/Parkinsson disease.csv"  # <- change if needed

if not os.path.exists(csv_path):
    raise FileNotFoundError(f"CSV file not found at {csv_path}. Upload the file to Colab or set the path correctly.")

df = pd.read_csv(csv_path)
print("Loaded CSV. Shape:", df.shape)
print("Columns before sanitizing:", df.columns.tolist())

# Sanitize column names for LightGBM
def sanitize_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'[^\w_]+', '', col) # Remove special characters
        new_cols.append(new_col)
    df.columns = new_cols
    return df

df = sanitize_col_names(df)
print("Columns after sanitizing:", df.columns.tolist())


print("\nHead:")
print(df.head().to_string())

# ---------- 4) Quick EDA ----------
print("\n--- Data types & missing counts ---")
print(df.info())
print(df.isnull().sum().sort_values(ascending=False).head(30))

# ---------- 5) Auto-detect columns ----------
lower_cols = [c.lower() for c in df.columns]

# status / classification target
status_col = None
for candidate in ['status','class','target','parkinsons','label','diagnosis','is_parkinson','pd']:
    if candidate in lower_cols:
        status_col = df.columns[lower_cols.index(candidate)]
        break

# updrs columns for regression
updrs_cols = [c for c in df.columns if 'updrs' in c.lower()]

# possible patient/group id column
group_col = None
for candidate in ['id','subject','subject#','patient_id','patient','name','file']:
    if candidate in lower_cols:
        group_col = df.columns[lower_cols.index(candidate)]
        break

print("\nDetected status (classification) column:", status_col)
print("Detected UPDRS columns (regression candidates):", updrs_cols)
print("Detected group/patient column:", group_col)

has_classification = status_col is not None
has_regression = len(updrs_cols) > 0

# ---------- 6) Feature set ----------
drop_candidates = []
if group_col: drop_candidates.append(group_col)
if status_col: drop_candidates.append(status_col)
drop_candidates += updrs_cols
# Also drop obvious id-like columns
for c in df.columns:
    if c.lower() in ['name','file']:
        drop_candidates.append(c)

feature_cols = [c for c in df.columns if c not in drop_candidates]
print("\nDetected feature columns count:", len(feature_cols))

# ---------- helper: preprocess ----------
def preprocess_features(X):
    # numeric impute and scaling; one-hot encode categoricals
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    imp = SimpleImputer(strategy='median')
    X_num = pd.DataFrame(imp.fit_transform(X[num_cols]), columns=num_cols, index=X.index)
    cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        X_cat = pd.get_dummies(X[cat_cols].fillna('NA'), drop_first=True)
        X_proc = pd.concat([X_num, X_cat], axis=1)
    else:
        X_proc = X_num
    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_proc), columns=X_proc.columns, index=X_proc.index)
    return X_scaled, imp, scaler

# ---------- 7) CLASSIFICATION pipeline ----------
results = {}
if has_classification:
    print("\n===== CLASSIFICATION PIPELINE =====")
    y = df[status_col]
    X = df[feature_cols]
    # groups for patient-level split if available
    groups = None
    if group_col:
        groups = df[group_col].astype(str).values

    X_proc, imp_clf, scaler_clf = preprocess_features(X)

    # Feature selection: use BorutaSHAP if available, else SelectKBest (mutual info)
    if BORUTA_AVAILABLE:
        print("Running BorutaSHAP (this can take time)...")
        try:
            # BorutaShap requires a model and a small background sample
            model_for_boruta = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
            boruta = BorutaShap(model=model_for_boruta, n_estimators='auto', sample=False, percentile=95, verbose=0)
            boruta.fit(X=X_proc, y=y, n_trials=100, random_state=42)
            selected = boruta.support
            sel_features = X_proc.columns[selected].tolist()
            if len(sel_features) == 0:
                raise Exception("BorutaSHAP selected zero features; fallback to SelectKBest")
        except Exception as e:
            print("BorutaSHAP failed or selected none; falling back to SelectKBest. Error:", e)
            BORUTA_AVAILABLE = False

    if not BORUTA_AVAILABLE:
        k = min(25, X_proc.shape[1])
        selector = SelectKBest(mutual_info_classif, k=k)
        selector.fit(X_proc.fillna(0), y)
        sel_features = X_proc.columns[selector.get_support()].tolist()

    print("Selected features for classification:", sel_features)
    X_sel = X_proc[sel_features]

    # Train/val/test split (patient-group aware if possible)
    if groups is not None:
        unique_groups = np.unique(groups)
        rng = np.random.RandomState(42); rng.shuffle(unique_groups)
        n = len(unique_groups)
        n_train = max(1, int(0.7 * n)); n_val = max(1, int(0.15 * n)); n_test = n - n_train - n_val
        train_g = unique_groups[:n_train]
        val_g = unique_groups[n_train:n_train+n_val]
        test_g = unique_groups[n_train+n_val:]
        train_idx = df[df[group_col].isin(train_g)].index
        val_idx = df[df[group_col].isin(val_g)].index
        test_idx = df[df[group_col].isin(test_g)].index
        X_train, y_train = X_sel.loc[train_idx], y.loc[train_idx]
        X_val, y_val = X_sel.loc[val_idx], y.loc[val_idx]
        X_test, y_test = X_sel.loc[test_idx], y.loc[test_idx]
    else:
        # stratified split
        X_train, X_temp, y_train, y_temp = train_test_split(X_sel, y, test_size=0.3, stratify=y, random_state=42)
        X_val, X_test, y_val, y_temp = train_test_split(X_temp, y_temp, test_size=0.5, stratify=y_temp, random_state=42)

    print("Train/Val/Test shapes:", X_train.shape, X_val.shape, X_test.shape)

    # Handle imbalance with SMOTE on training data
    try:
        sm = SMOTE(random_state=42)
        X_train_sm, y_train_sm = sm.fit_resample(X_train, y_train)
    except Exception as e:
        print("SMOTE failed, using original train set. Error:", e)
        X_train_sm, y_train_sm = X_train, y_train

    # Models to evaluate
    clf_models = {
        "RandomForest": RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric='logloss', n_estimators=200, random_state=42),
        "LightGBM": lgb.LGBMClassifier(n_estimators=200, random_state=42),
        "SVM": SVC(probability=True, random_state=42)
    }

    clf_results = {}
    for name, model in clf_models.items():
        print(f"Training {name}...")
        model.fit(X_train_sm, y_train_sm)
        y_pred = model.predict(X_test)
        y_proba = model.predict_proba(X_test)[:,1] if hasattr(model, "predict_proba") else None
        acc = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        prec = precision_score(y_test, y_pred, zero_division=0)
        rec = recall_score(y_test, y_pred, zero_division=0)
        roc = roc_auc_score(y_test, y_proba) if y_proba is not None else None
        clf_results[name] = {"model": model, "accuracy": acc, "f1": f1, "precision": prec, "recall": rec, "roc_auc": roc}
        print(f"  {name} — Acc: {acc:.4f}, F1: {f1:.4f}, Prec: {prec:.4f}, Rec: {rec:.4f}, ROC-AUC: {roc if roc is not None else 'N/A'}")

    # Choose best classifier by F1
    best_clf_name = max(clf_results.keys(), key=lambda k: clf_results[k]['f1'])
    best_clf = clf_results[best_clf_name]['model']
    print("\nBest classifier by F1:", best_clf_name)

    # Save confusion matrix & ROC plot
    y_pred_best = best_clf.predict(X_test)
    y_proba_best = best_clf.predict_proba(X_test)[:,1] if hasattr(best_clf, "predict_proba") else None

    cm = confusion_matrix(y_test, y_pred_best)
    plt.figure(figsize=(5,4)); plt.imshow(cm, interpolation='nearest'); plt.title(f'Confusion Matrix - {best_clf_name}'); plt.xlabel('Predicted'); plt.ylabel('True'); plt.colorbar(); plt.tight_layout()
    cm_path = "/content/confusion_matrix_classification.png"
    plt.savefig(cm_path); plt.close()
    print("Saved confusion matrix to", cm_path)

    if y_proba_best is not None:
        from sklearn.metrics import roc_curve, auc
        fpr, tpr, _ = roc_curve(y_test, y_proba_best)
        roc_auc = auc(fpr, tpr)
        plt.figure(figsize=(6,4)); plt.plot(fpr, tpr); plt.plot([0,1],[0,1],'--'); plt.title(f'ROC Curve - {best_clf_name} (AUC={roc_auc:.3f})'); plt.xlabel('FPR'); plt.ylabel('TPR'); plt.tight_layout()
        roc_path = "/content/roc_curve_classification.png"
        plt.savefig(roc_path); plt.close()
        print("Saved ROC curve to", roc_path)

    # Save model
    clf_model_path = "/content/best_classifier.joblib"
    joblib.dump(best_clf, clf_model_path)
    print("Saved best classifier to", clf_model_path)

    results['classification'] = {"results": clf_results, "best": best_clf_name, "model_path": clf_model_path, "features": sel_features}

# ---------- 8) REGRESSION pipeline (UPDRS) ----------
if has_regression:
    print("\n===== REGRESSION PIPELINE =====")
    # choose the most appropriate UPDRS column (prefer 'total' or 'motor' if present)
    target_reg = None
    for c in updrs_cols:
        if 'total' in c.lower() or 'motor' in c.lower():
            target_reg = c; break
    if target_reg is None:
        target_reg = updrs_cols[0]
    print("Regression target chosen:", target_reg)

    y_reg = df[target_reg]
    X_reg = df[feature_cols]
    X_reg_proc, imp_reg, scaler_reg = preprocess_features(X_reg)

    # Feature selection (mutual_info_regression)
    k = min(25, X_reg_proc.shape[1])
    selector_reg = SelectKBest(mutual_info_regression, k=k)
    selector_reg.fit(X_reg_proc.fillna(0), y_reg.fillna(y_reg.median()))
    sel_features_reg = X_reg_proc.columns[selector_reg.get_support()].tolist()
    print("Selected features for regression:", sel_features_reg)
    X_reg_sel = X_reg_proc[sel_features_reg]

    # Split train/val/test
    if group_col is not None:
        unique_groups = np.unique(df[group_col].astype(str).values)
        rng = np.random.RandomState(42); rng.shuffle(unique_groups)
        n = len(unique_groups)
        n_train = max(1, int(0.7*n)); n_val = max(1, int(0.15*n)); n_test = n - n_train - n_val
        train_g = unique_groups[:n_train]; val_g = unique_groups[n_train:n_train+n_val]; test_g = unique_groups[n_train+n_val:]
        train_idx = df[df[group_col].isin(train_g)].index
        val_idx = df[df[group_col].isin(val_g)].index
        test_idx = df[df[group_col].isin(test_g)].index
        X_train_r, y_train_r = X_reg_sel.loc[train_idx], y_reg.loc[train_idx]
        X_val_r, y_val_r = X_reg_sel.loc[val_idx], y_reg.loc[val_idx]
        X_test_r, y_test_r = X_reg_sel.loc[test_idx], y_reg.loc[test_idx]
    else:
        X_train_r, X_temp_r, y_train_r, y_temp_r = train_test_split(X_reg_sel, y_reg, test_size=0.3, random_state=42)
        X_val_r, X_test_r, y_val_r, y_test_r = train_test_split(X_temp_r, y_temp_r, test_size=0.5, random_state=42)

    print("Regression Train/Val/Test shapes:", X_train_r.shape, X_val_r.shape, X_test_r.shape)

    reg_models = {
        "Linear": LinearRegression(),
        "ElasticNet": ElasticNet(random_state=42),
        "RandomForest": RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
        "XGBoost": XGBRegressor(n_estimators=200, random_state=42),
        "LightGBM": lgb.LGBMRegressor(n_estimators=200, random_state=42)
    }

    reg_results = {}
    for name, model in reg_models.items():
        print(f"Training regressor {name}...")
        model.fit(X_train_r, y_train_r)
        y_pred_r = model.predict(X_test_r)
        mae = mean_absolute_error(y_test_r, y_pred_r)
        rmse = mean_squared_error(y_test_r, y_pred_r, squared=False)
        r2 = r2_score(y_test_r, y_pred_r)
        reg_results[name] = {"model": model, "MAE": mae, "RMSE": rmse, "R2": r2}
        print(f"  {name} — MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

    best_reg_name = min(reg_results.keys(), key=lambda k: reg_results[k]['MAE'])
    best_reg = reg_results[best_reg_name]['model']
    print("Best regressor by MAE:", best_reg_name)

    # Save scatter predicted vs actual
    y_pred_best_r = best_reg.predict(X_test_r)
    plt.figure(figsize=(6,5)); plt.scatter(y_test_r, y_pred_best_r); plt.xlabel("Actual "+target_reg); plt.ylabel("Predicted "+target_reg); plt.title(f"Predicted vs Actual - {best_reg_name}"); plt.tight_layout()
    scatter_path = "/content/predicted_vs_actual_regression.png"
    plt.savefig(scatter_path); plt.close()
    print("Saved regression scatter to", scatter_path)

    reg_model_path = "/content/best_regressor.joblib"
    joblib.dump(best_reg, reg_model_path)
    print("Saved best regressor to", reg_model_path)

    results['regression'] = {"results": reg_results, "best": best_reg_name, "model_path": reg_model_path, "features": sel_features_reg}

# ---------- 9) SHAP explainability (approx) ----------
if SHAP_AVAILABLE:
    try:
        if 'classification' in results:
            best_name = results['classification']['best']
            best_model = results['classification']['results'][best_name]['model']
            # Only do SHAP for tree-based models reliably
            if hasattr(best_model, "feature_importances_"):
                X_for_shap = X_sel if 'X_sel' in locals() else X_proc
                explainer = shap.TreeExplainer(best_model)
                shap_values = explainer.shap_values(X_for_shap)
                # plot mean absolute shap per feature as bar (safe and portable)
                if isinstance(shap_values, list):
                    vals = np.abs(shap_values[1]).mean(axis=0)
                else:
                    vals = np.abs(shap_values).mean(axis=0)
                order = np.argsort(vals)[::-1][:20]
                plt.figure(figsize=(8,6)); plt.barh(X_for_shap.columns[order][::-1], vals[order][::-1]); plt.xlabel("Mean |SHAP value|"); plt.title("Global feature importance - Classification"); plt.tight_layout()
                shap_clf_path = "/content/shap_classification_bar.png"; plt.savefig(shap_clf_path); plt.close()
                print("Saved classification SHAP bar to", shap_clf_path)

        if 'regression' in results:
            best_reg_name = results['regression']['best']
            best_reg_model = results['regression']['results'][best_reg_name]['model']
            if hasattr(best_reg_model, "feature_importances_"):
                X_for_shap_r = X_reg_sel if 'X_reg_sel' in locals() else X_reg_proc
                explainer_r = shap.TreeExplainer(best_reg_model)
                shap_values_r = explainer_r.shap_values(X_for_shap_r)
                if isinstance(shap_values_r, list):
                    valsr = np.abs(shap_values_r).mean(axis=0)
                else:
                    valsr = np.abs(shap_values_r).mean(axis=0)
                orderr = np.argsort(valsr)[::-1][:20]
                plt.figure(figsize=(8,6)); plt.barh(X_for_shap_r.columns[orderr][::-1], valsr[orderr][::-1]); plt.xlabel("Mean |SHAP value|"); plt.title("Global feature importance - Regression"); plt.tight_layout()
                shap_reg_path = "/content/shap_regression_bar.png"; plt.savefig(shap_reg_path); plt.close()
                print("Saved regression SHAP bar to", shap_reg_path)

    except Exception as e:
        print("SHAP step failed or partially failed:", e)
else:
    print("SHAP not installed; skip SHAP explainability. In Colab, consider installing 'shap' to enable this section.")

# ---------- 10) Save a summary CSV ----------
summary_rows = []
if 'classification' in results:
    for name, v in results['classification']['results'].items():
        summary_rows.append({"task":"classification","model":name,"accuracy":v['accuracy'],"f1":v['f1'],"precision":v['precision'],"recall":v['recall'],"roc_auc":v['roc_auc']})
if 'regression' in results:
    for name, v in results['regression']['results'].items():
        summary_rows.append({"task":"regression","model":name,"MAE":v['MAE'],"RMSE":v['RMSE'],"R2":v['R2']})
summary_df = pd.DataFrame(summary_rows)
summary_csv = "/content/model_results_summary.csv"
summary_df.to_csv(summary_csv, index=False)
print("Saved summary CSV to", summary_csv)

# ---------- 11) Final printout ----------
print("\n--- Completed. Files saved to /content ---")
for f in os.listdir("/content"):
    if any(ext in f for ext in [".png", ".joblib", ".csv"]):
        print(" -", f)

print("\nIf you want a downloadable .ipynb or modifications (grid search, LOSO CV, more explainability plots), tell me which part to expand and I will produce the code snippet.")

ERROR: Could not find a version that satisfies the requirement boruta_shap (from versions: none)
ERROR: No matching distribution found for boruta_shap
Loaded CSV. Shape: (195, 24)
Columns before sanitizing: ['name', 'MDVP:Fo(Hz)', 'MDVP:Fhi(Hz)', 'MDVP:Flo(Hz)', 'MDVP:Jitter(%)', 'MDVP:Jitter(Abs)', 'MDVP:RAP', 'MDVP:PPQ', 'Jitter:DDP', 'MDVP:Shimmer', 'MDVP:Shimmer(dB)', 'Shimmer:APQ3', 'Shimmer:APQ5', 'MDVP:APQ', 'Shimmer:DDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']
Columns after sanitizing: ['name', 'MDVPFoHz', 'MDVPFhiHz', 'MDVPFloHz', 'MDVPJitter', 'MDVPJitterAbs', 'MDVPRAP', 'MDVPPPQ', 'JitterDDP', 'MDVPShimmer', 'MDVPShimmerdB', 'ShimmerAPQ3', 'ShimmerAPQ5', 'MDVPAPQ', 'ShimmerDDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']

Head:
             name  MDVPFoHz  MDVPFhiHz  MDVPFloHz  MDVPJitter  MDVPJitterAbs  MDVPRAP  MDVPPPQ  JitterDDP  MDVPShimmer  MDVPShimmerdB  ShimmerAPQ3  ShimmerAPQ5  MDVPAPQ  ShimmerDDA     

In [20]:
# =============================================================================
# COMPREHENSIVE PARKINSON'S DISEASE DETECTION PIPELINE WITH ENHANCED VISUALIZATIONS
# =============================================================================

# ---------- 0) Environment Setup ----------
!pip install -q pandas numpy scikit-learn matplotlib seaborn xgboost lightgbm shap imbalanced-learn joblib scipy plotly
try:
    !pip install -q boruta_shap
    BORUTA_SHAP_INSTALLED = True
except Exception:
    BORUTA_SHAP_INSTALLED = False

# ---------- 1) Imports ----------
import os
import sys
import warnings
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import joblib
import re
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, confusion_matrix, classification_report,
                           mean_absolute_error, mean_squared_error, r2_score,
                           roc_curve, auc, precision_recall_curve)
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from xgboost import XGBClassifier, XGBRegressor
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
import shap

warnings.filterwarnings("ignore")

# Set style for publication-quality figures
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

print("Imports completed successfully")
print(f"SHAP version: {shap.__version__}")

# ---------- 2) Create Output Directory Structure ----------
output_dirs = ['/content/figures', '/content/tables', '/content/models', '/content/results']
for dir_path in output_dirs:
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created directory: {dir_path}")

# ---------- 3) Data Loading and Exploration ----------
print("\n" + "="*80)
print("3. DATA LOADING AND EXPLORATION")
print("="*80)

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/ML COURSE PAPER/DATASET/Parkinsson disease.csv')
print(f"Dataset shape: {df.shape}")

# Sanitize column names for LightGBM compatibility
def sanitize_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'[^\w_]+', '', col) # Remove special characters
        new_cols.append(new_col)
    df.columns = new_cols
    return df

df = sanitize_col_names(df)
print("Columns after sanitizing:", df.columns.tolist())


# Basic dataset info
dataset_info = {
    'Total Samples': len(df),
    'Total Features': len(df.columns) - 2,  # excluding name and status
    'Parkinson Patients': df['status'].sum(),
    'Healthy Controls': len(df) - df['status'].sum(),
    'Parkinson Percentage': f"{df['status'].sum()/len(df)*100:.1f}%",
    'Healthy Percentage': f"{(len(df) - df['status'].sum())/len(df)*100:.1f}%"
}

print("\nDataset Summary:")
for key, value in dataset_info.items():
    print(f"{key}: {value}")

# Extract patient IDs
df['patient_id'] = df['name'].str.extract(r'(phon_R\d+_S\d+)')
print(f"Unique patients: {df['patient_id'].nunique()}")

# ---------- 4) Enhanced Data Visualization ----------
print("\n" + "="*80)
print("4. ENHANCED DATA VISUALIZATION")
print("="*80)

# 4.1 Class Distribution Pie Chart
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
class_counts = df['status'].value_counts()
plt.pie(class_counts, labels=['Healthy', 'Parkinson\'s'], autopct='%1.1f%%',
        colors=['lightgreen', 'lightcoral'], startangle=90)
plt.title('Class Distribution')

# 4.2 Feature Distribution by Class
plt.subplot(1, 2, 2)
# Select a representative feature for visualization
feature_to_plot = 'MDVPFoHz' if 'MDVPFoHz' in df.columns else df.columns[1]
if feature_to_plot in df.columns:
    healthy_vals = df[df['status'] == 0][feature_to_plot]
    parkinson_vals = df[df['status'] == 1][feature_to_plot]
    plt.hist([healthy_vals, parkinson_vals], bins=20, alpha=0.7,
             label=['Healthy', 'Parkinson\'s'], color=['green', 'red'])
    plt.xlabel(feature_to_plot)
    plt.ylabel('Frequency')
    plt.legend()
    plt.title(f'Distribution of {feature_to_plot}')

plt.tight_layout()
plt.savefig('/content/figures/class_distribution.png', bbox_inches='tight', dpi=300)
plt.savefig('/content/figures/class_distribution.pdf', bbox_inches='tight')
plt.close()

# 4.3 Correlation Matrix Heatmap
plt.figure(figsize=(16, 14))
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_matrix = df[numeric_cols].corr()

# Create mask for upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8},
            annot=False, fmt=".2f")
plt.title('Feature Correlation Matrix', fontsize=14, pad=20)
plt.tight_layout()
plt.savefig('/content/figures/correlation_matrix.png', bbox_inches='tight', dpi=300)
plt.savefig('/content/figures/correlation_matrix.pdf', bbox_inches='tight')
plt.close()

# 4.4 PCA Visualization
plt.figure(figsize=(12, 5))

# Prepare data for PCA
X_pca = df.drop(['name', 'status', 'patient_id'], axis=1, errors='ignore')
X_pca = X_pca.select_dtypes(include=[np.number])
X_pca = X_pca.fillna(X_pca.mean())

# Standardize
scaler_pca = StandardScaler()
X_pca_scaled = scaler_pca.fit_transform(X_pca)

# Apply PCA
pca = PCA(n_components=2)
X_pca_reduced = pca.fit_transform(X_pca_scaled)

plt.subplot(1, 2, 1)
scatter = plt.scatter(X_pca_reduced[:, 0], X_pca_reduced[:, 1],
                     c=df['status'], cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.title('PCA - Parkinson vs Healthy')

# 4.5 t-SNE Visualization
plt.subplot(1, 2, 2)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_pca_scaled)

scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1],
                     c=df['status'], cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title('t-SNE Visualization')

plt.tight_layout()
plt.savefig('/content/figures/dimensionality_reduction.png', bbox_inches='tight', dpi=300)
plt.savefig('/content/figures/dimensionality_reduction.pdf', bbox_inches='tight')
plt.close()

print("Enhanced data visualizations saved")

# ---------- 5) Data Preprocessing ----------
print("\n" + "="*80)
print("5. DATA PREPROCESSING")
print("="*80)

# Prepare features and target
X = df.drop(['name', 'status', 'patient_id'], axis=1, errors='ignore')
y = df['status']

# Handle missing values
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Standardize features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)

print("Data preprocessing completed")
print(f"Final feature set: {X_scaled.shape[1]} features")

# ---------- 6) Feature Selection ----------
print("\n" + "="*80)
print("6. FEATURE SELECTION")
print("="*80)

# Select top features using mutual information
selector = SelectKBest(mutual_info_classif, k=min(20, X_scaled.shape[1]))
X_selected = selector.fit_transform(X_scaled, y)
selected_features = X_scaled.columns[selector.get_support()].tolist()

print(f"Selected {len(selected_features)} features:")
for i, feature in enumerate(selected_features, 1):
    print(f"{i:2d}. {feature}")

# Save feature selection results
feature_importance_df = pd.DataFrame({
    'Feature': X_scaled.columns,
    'Importance': selector.scores_,
    'Selected': selector.get_support()
}).sort_values('Importance', ascending=False)

feature_importance_df.to_csv('/content/tables/feature_importance.csv', index=False)

# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance_df.head(15)
sns.barplot(data=top_features, x='Importance', y='Feature')
plt.title('Top 15 Most Important Features (Mutual Information)')
plt.tight_layout()
plt.savefig('/content/figures/feature_importance.png', bbox_inches='tight', dpi=300)
plt.savefig('/content/figures/feature_importance.pdf', bbox_inches='tight')
plt.close()

# ---------- 7) Patient-Level Data Splitting ----------
print("\n" + "="*80)
print("7. PATIENT-LEVEL DATA SPLITTING")
print("="*80)

# Get unique patients
unique_patients = df['patient_id'].unique()
patient_status = df.groupby('patient_id')['status'].first()

# Stratified split by patient
train_patients, temp_patients = train_test_split(
    unique_patients, test_size=0.3, random_state=42,
    stratify=patient_status
)
val_patients, test_patients = train_test_split(
    temp_patients, test_size=0.5, random_state=42,
    stratify=patient_status[patient_status.index.isin(temp_patients)]
)

# Create masks
train_mask = df['patient_id'].isin(train_patients)
val_mask = df['patient_id'].isin(val_patients)
test_mask = df['patient_id'].isin(test_patients)

# Apply splits
X_train = X_scaled[train_mask]
X_val = X_scaled[val_mask]
X_test = X_scaled[test_mask]
y_train = y[train_mask]
y_val = y[val_mask]
y_test = y[test_mask]

# Use only selected features
X_train_sel = X_train[selected_features]
X_val_sel = X_val[selected_features]
X_test_sel = X_test[selected_features]

print(f"Training set: {X_train_sel.shape[0]} samples ({len(train_patients)} patients)")
print(f"Validation set: {X_val_sel.shape[0]} samples ({len(val_patients)} patients)")
print(f"Test set: {X_test_sel.shape[0]} samples ({len(test_patients)} patients)")

# Save split information
split_info = pd.DataFrame({
    'patient_id': df['patient_id'],
    'split': ['train' if m else 'val' if v else 'test'
              for m, v, t in zip(train_mask, val_mask, test_mask)]
})
split_info.to_csv('/content/tables/data_splits.csv', index=False)

# ---------- 8) Handle Class Imbalance with SMOTE ----------
print("\n" + "="*80)
print("8. HANDLING CLASS IMBALANCE")
print("="*80)

print(f"Before SMOTE - Class distribution: {pd.Series(y_train).value_counts().to_dict()}")

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_sel, y_train)

print(f"After SMOTE - Class distribution: {pd.Series(y_train_smote).value_counts().to_dict()}")

# ---------- 9) Model Training and Evaluation ----------
print("\n" + "="*80)
print("9. MODEL TRAINING AND EVALUATION")
print("="*80)

# Define models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
    'Support Vector Machine': SVC(probability=True, random_state=42),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss', n_estimators=200),
    'LightGBM': lgb.LGBMClassifier(random_state=42, n_estimators=200),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=200)
}

# Train and evaluate models
model_results = {}
trained_models = {}

for name, model in models.items():
    print(f"\nTraining {name}...")

    # Train model
    model.fit(X_train_smote, y_train_smote)
    trained_models[name] = model

    # Predictions
    y_pred = model.predict(X_test_sel)
    y_pred_proba = model.predict_proba(X_test_sel)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Cross-validation scores
    cv_scores = cross_val_score(model, X_train_smote, y_train_smote,
                               cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()

    model_results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'cv_mean': cv_mean,
        'cv_std': cv_std,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

    print(f"  Accuracy: {accuracy:.4f}, F1: {f1:.4f}, AUC: {roc_auc:.4f}")
    print(f"  CV Accuracy: {cv_mean:.4f} ± {cv_std:.4f}")

# ---------- 10) Comprehensive Model Visualization ----------
print("\n" + "="*80)
print("10. COMPREHENSIVE MODEL VISUALIZATION")
print("="*80)

# 10.1 Performance Comparison Bar Chart
metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']

plt.figure(figsize=(15, 10))
for i, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
    plt.subplot(2, 3, i+1)
    values = [model_results[model][metric] for model in models.keys()]
    bars = plt.barh(list(models.keys()), values, color=plt.cm.Set3(np.linspace(0, 1, len(models))))
    plt.xlabel(metric_name)
    plt.xlim(0, 1)
    # Add value labels on bars
    for bar, value in zip(bars, values):
        plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                f'{value:.3f}', ha='left', va='center', fontsize=8)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/content/figures/model_performance_comparison.png', bbox_inches='tight', dpi=300)
plt.savefig('/content/figures/model_performance_comparison.pdf', bbox_inches='tight')
plt.close()

# 10.2 Confusion Matrices
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, (name, results) in enumerate(model_results.items()):
    cm = confusion_matrix(y_test, results['y_pred'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                xticklabels=['Healthy', 'Parkinson\'s'],
                yticklabels=['Healthy', 'Parkinson\'s'])
    axes[i].set_title(f'{name}\nAccuracy: {results["accuracy"]:.3f}', fontsize=12)
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('Actual')

# Remove empty subplot if needed
if len(models) < 6:
    for j in range(len(models), 6):
        axes[j].axis('off')

plt.tight_layout()
plt.savefig('/content/figures/confusion_matrices.png', bbox_inches='tight', dpi=300)
plt.savefig('/content/figures/confusion_matrices.pdf', bbox_inches='tight')
plt.close()

# 10.3 ROC Curves
plt.figure(figsize=(10, 8))
for name, results in model_results.items():
    fpr, tpr, _ = roc_curve(y_test, results['y_pred_proba'])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.5)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/content/figures/roc_curves.png', bbox_inches='tight', dpi=300)
plt.savefig('/content/figures/roc_curves.pdf', bbox_inches='tight')
plt.close()

# 10.4 Precision-Recall Curves
plt.figure(figsize=(10, 8))
for name, results in model_results.items():
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, results['y_pred_proba'])
    plt.plot(recall_vals, precision_vals, lw=2, label=f'{name}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend(loc="upper right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig('/content/figures/precision_recall_curves.png', bbox_inches='tight', dpi=300)
plt.savefig('/content/figures/precision_recall_curves.pdf', bbox_inches='tight')
plt.close()

# ---------- 11) SHAP Explainable AI ----------
print("\n" + "="*80)
print("11. SHAP EXPLAINABLE AI ANALYSIS")
print("="*80)

# Use the best model for SHAP analysis
best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['roc_auc'])
best_model = model_results[best_model_name]['model']
print(f"Using {best_model_name} for SHAP analysis (AUC: {model_results[best_model_name]['roc_auc']:.4f})")

try:
    # Create SHAP explainer
    explainer = shap.TreeExplainer(best_model)

    # Calculate SHAP values
    shap_values = explainer.shap_values(X_test_sel)

    # Handle binary classification output
    if isinstance(shap_values, list) and len(shap_values) == 2:
        shap_values = shap_values[1]  # Use positive class

    # 11.1 SHAP Summary Plot
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_test_sel, feature_names=selected_features, show=False)
    plt.tight_layout()
    plt.savefig('/content/figures/shap_summary.png', bbox_inches='tight', dpi=300)
    plt.savefig('/content/figures/shap_summary.pdf', bbox_inches='tight')
    plt.close()

    # 11.2 SHAP Bar Plot (Mean Absolute SHAP Values)
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_test_sel, feature_names=selected_features,
                     plot_type="bar", show=False)
    plt.tight_layout()
    plt.savefig('/content/figures/shap_bar_plot.png', bbox_inches='tight', dpi=300)
    plt.savefig('/content/figures/shap_bar_plot.pdf', bbox_inches='tight')
    plt.close()

    # 11.3 SHAP Force Plot for a specific instance
    plt.figure(figsize=(12, 4))
    # Check if expected_value is a list (multi-output) and use the positive class expected value
    expected_value = explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value
    shap.force_plot(expected_value, shap_values[0], X_test_sel.iloc[0], feature_names=selected_features,
                   matplotlib=True, show=False)
    plt.tight_layout()
    plt.savefig('/content/figures/shap_force_plot.png', bbox_inches='tight', dpi=300)
    plt.savefig('/content/figures/shap_force_plot.pdf', bbox_inches='tight')
    plt.close()

    print("SHAP analysis completed successfully")

except Exception as e:
    print(f"SHAP analysis failed: {e}")
    print("Proceeding without SHAP visualizations")

# ---------- 12) Create Combined Results Dashboard ----------
print("\n" + "="*80)
print("12. CREATING COMBINED RESULTS DASHBOARD")
print("="*80)

# 12.1 Create comprehensive results table
results_summary = []
for name, results in model_results.items():
    results_summary.append({
        'Model': name,
        'Accuracy': f"{results['accuracy']:.4f}",
        'Precision': f"{results['precision']:.4f}",
        'Recall': f"{results['recall']:.4f}",
        'F1-Score': f"{results['f1_score']:.4f}",
        'ROC-AUC': f"{results['roc_auc']:.4f}",
        'CV Accuracy (Mean)': f"{results['cv_mean']:.4f}",
        'CV Accuracy (Std)': f"{results['cv_std']:.4f}"
    })

results_df = pd.DataFrame(results_summary)
results_df.to_csv('/content/tables/comprehensive_results.csv', index=False)
results_df.to_excel('/content/tables/comprehensive_results.xlsx', index=False)

# 12.2 Create a summary visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Model performance comparison
metrics_plot = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
x_pos = np.arange(len(models))
width = 0.15

for i, metric in enumerate(metrics_plot):
    values = [model_results[model][metric] for model in models.keys()]
    ax1.bar(x_pos + i*width, values, width, label=metric.replace('_', ' ').title())

ax1.set_xlabel('Models')
ax1.set_ylabel('Score')
ax1.set_title('Model Performance Comparison')
ax1.set_xticks(x_pos + width*2)
ax1.set_xticklabels(list(models.keys()), rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# ROC Curves
for name, results in model_results.items():
    fpr, tpr, _ = roc_curve(y_test, results['y_pred_proba'])
    roc_auc = auc(fpr, tpr)
    ax2.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.3f})')

ax2.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title('ROC Curves')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Feature Importance
ax3.barh(range(len(top_features)), top_features['Importance'])
ax3.set_yticks(range(len(top_features)))
ax3.set_yticklabels(top_features['Feature'])
ax3.set_xlabel('Importance Score')
ax3.set_title('Top Feature Importance')
ax3.grid(True, alpha=0.3)

# Class Distribution
class_counts = [dataset_info['Healthy Controls'], dataset_info['Parkinson Patients']]
ax4.pie(class_counts, labels=['Healthy', 'Parkinson\'s'], autopct='%1.1f%%',
        colors=['lightgreen', 'lightcoral'], startangle=90)
ax4.set_title('Class Distribution')

plt.tight_layout()
plt.savefig('/content/figures/combined_results_dashboard.png', bbox_inches='tight', dpi=300)
plt.savefig('/content/figures/combined_results_dashboard.pdf', bbox_inches='tight')
plt.close()

# ---------- 13) Save Models and Preprocessors ----------
print("\n" + "="*80)
print("13. SAVING MODELS AND PREPROCESSORS")
print("="*80)

# Save all trained models
for name, model in trained_models.items():
    joblib.dump(model, f'/content/models/model_{name.replace(" ", "_").lower()}.pkl')

# Save preprocessors
joblib.dump(scaler, '/content/models/scaler.pkl')
joblib.dump(imputer, '/content/models/imputer.pkl')
joblib.dump(selector, '/content/models/feature_selector.pkl')

print("All models and preprocessors saved successfully")

# ---------- 14) Generate Final Report ----------
print("\n" + "="*80)
print("14. GENERATING FINAL REPORT")
print("="*80)

# Create a comprehensive report
report_content = f"""
PARKINSON'S DISEASE DETECTION - COMPREHENSIVE ANALYSIS REPORT
================================================================================

EXPERIMENT SUMMARY:
- Dataset: Parkinsson disease.csv
- Total Samples: {dataset_info['Total Samples']}
- Features: {dataset_info['Total Features']}
- Parkinson's Patients: {dataset_info['Parkinson Patients']} ({dataset_info['Parkinson Percentage']})
- Healthy Controls: {dataset_info['Healthy Controls']} ({dataset_info['Healthy Percentage']})
- Unique Patients: {df['patient_id'].nunique()}

DATA SPLITS:
- Training Set: {X_train_sel.shape[0]} samples
- Validation Set: {X_val_sel.shape[0]} samples
- Test Set: {X_test_sel.shape[0]} samples

BEST PERFORMING MODEL: {best_model_name}
- Test Accuracy: {model_results[best_model_name]['accuracy']:.4f}
- Test F1-Score: {model_results[best_model_name]['f1_score']:.4f}
- Test ROC-AUC: {model_results[best_model_name]['roc_auc']:.4f}
- CV Accuracy: {model_results[best_model_name]['cv_mean']:.4f} ± {model_results[best_model_name]['cv_std']:.4f}

TOP 5 FEATURES:
{top_features[['Feature', 'Importance']].head().to_string(index=False)}

MODEL PERFORMANCE RANKING (by ROC-AUC):
"""

# Add model ranking
ranking = sorted(model_results.items(), key=lambda x: x[1]['roc_auc'], reverse=True)
for i, (name, results) in enumerate(ranking, 1):
    report_content += f"{i}. {name}: AUC = {results['roc_auc']:.4f}, F1 = {results['f1_score']:.4f}\n"

report_content += f"""
FILES GENERATED:

FIGURES:
- class_distribution.png/pdf - Class distribution and feature visualization
- correlation_matrix.png/pdf - Feature correlation heatmap
- dimensionality_reduction.png/pdf - PCA and t-SNE visualizations
- feature_importance.png/pdf - Feature importance ranking
- model_performance_comparison.png/pdf - Model comparison across metrics
- confusion_matrices.png/pdf - Confusion matrices for all models
- roc_curves.png/pdf - ROC curves for model comparison
- precision_recall_curves.png/pdf - Precision-Recall curves
- shap_*.png/pdf - SHAP explainability plots
- combined_results_dashboard.png/pdf - Comprehensive results summary

TABLES:
- feature_importance.csv - Complete feature importance scores
- data_splits.csv - Train/validation/test split information
- comprehensive_results.csv/xlsx - Detailed model performance metrics

MODELS:
- model_*.pkl - All trained machine learning models
- preprocessors.pkl - Scaler, imputer, and feature selector

This analysis provides a comprehensive evaluation of machine learning models for Parkinson's disease detection using voice features.
All results are suitable for conference paper submission.
"""

# Save report
with open('/content/results/comprehensive_analysis_report.txt', 'w') as f:
    f.write(report_content)

print(report_content)

# ---------- 15) Final Summary ----------
print("\n" + "="*80)
print("EXPERIMENT COMPLETED SUCCESSFULLY!")
print("="*80)

best_model = max(model_results.items(), key=lambda x: x[1]['roc_auc'])
print(f"\nBEST PERFORMING MODEL: {best_model[0]}")
print(f"ROC-AUC: {model_results[best_model[0]]['roc_auc']:.4f}")
print(f"Accuracy: {model_results[best_model[0]]['accuracy']:.4f}")
print(f"F1-Score: {model_results[best_model[0]]['f1_score']:.4f}")

print(f"\nALL OUTPUTS SAVED IN:")
print("📊 /content/figures/ - All visualizations (PNG/PDF)")
print("📋 /content/tables/ - All data tables (CSV/XLSX)")
print("🤖 /content/models/ - Trained models and preprocessors")
print("📄 /content/results/ - Comprehensive analysis report")

print(f"\n🎯 READY FOR CONFERENCE PAPER SUBMISSION!")

ERROR: Could not find a version that satisfies the requirement boruta_shap (from versions: none)
ERROR: No matching distribution found for boruta_shap
Imports completed successfully
SHAP version: 0.48.0
Created directory: /content/figures
Created directory: /content/tables
Created directory: /content/models
Created directory: /content/results

3. DATA LOADING AND EXPLORATION
Dataset shape: (195, 24)
Columns after sanitizing: ['name', 'MDVPFoHz', 'MDVPFhiHz', 'MDVPFloHz', 'MDVPJitter', 'MDVPJitterAbs', 'MDVPRAP', 'MDVPPPQ', 'JitterDDP', 'MDVPShimmer', 'MDVPShimmerdB', 'ShimmerAPQ3', 'ShimmerAPQ5', 'MDVPAPQ', 'ShimmerDDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']

Dataset Summary:
Total Samples: 195
Total Features: 22
Parkinson Patients: 147
Healthy Controls: 48
Parkinson Percentage: 75.4%
Healthy Percentage: 24.6%
Unique patients: 32

4. ENHANCED DATA VISUALIZATION
Enhanced data visualizations saved

5. DATA PREPROCESSING
Data preprocessing completed
Fin

<Figure size 3000x2400 with 0 Axes>

<Figure size 3000x2400 with 0 Axes>

<Figure size 3600x1200 with 0 Axes>

In [21]:
# =============================================================================
# COMPREHENSIVE PARKINSON'S DISEASE DETECTION PIPELINE
# Output saved to: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/
# =============================================================================

# ---------- 1) Environment setup ----------
!pip install -q pandas numpy scikit-learn matplotlib seaborn xgboost lightgbm shap imbalanced-learn joblib scipy plotly

# Optional packages
try:
    !pip install -q boruta_shap
    BORUTA_AVAILABLE = True
except Exception as e:
    print("boruta_shap install failed; falling back to SelectKBest. Error:", e)
    BORUTA_AVAILABLE = False

# ---------- 2) Imports ----------
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import re
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, mutual_info_classif, mutual_info_regression, RFE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, confusion_matrix, classification_report,
                           mean_absolute_error, mean_squared_error, r2_score,
                           roc_curve, auc, precision_recall_curve)
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from xgboost import XGBClassifier, XGBRegressor
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
import shap

warnings.filterwarnings("ignore")

# Set style for publication-quality figures
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

print("Imports completed successfully")

# ---------- 3) Create Output Directory Structure ----------
# Create timestamp for unique run identification
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_base = f"/content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_{timestamp}"

output_dirs = {
    'figures': f"{output_base}/figures",
    'tables': f"{output_base}/tables",
    'models': f"{output_base}/models",
    'results': f"{output_base}/results",
    'shap': f"{output_base}/shap"
}

for dir_name, dir_path in output_dirs.items():
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created directory: {dir_path}")

# ---------- 4) Data Loading and Sanitization ----------
print("\n" + "="*80)
print("4. DATA LOADING AND SANITIZATION")
print("="*80)

# Load dataset
csv_path = "/content/drive/MyDrive/ML COURSE PAPER/DATASET/Parkinsson disease.csv"
df = pd.read_csv(csv_path)
print(f"Dataset shape: {df.shape}")

# Sanitize column names
def sanitize_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'[^\w_]+', '', col)
        new_cols.append(new_col)
    df.columns = new_cols
    return df

df = sanitize_col_names(df)
print("Sanitized columns:", df.columns.tolist())

# ---------- 5) Auto-detect Columns ----------
lower_cols = [c.lower() for c in df.columns]

# Classification target
status_col = None
for candidate in ['status','class','target','parkinsons','label','diagnosis','is_parkinson','pd']:
    if candidate in lower_cols:
        status_col = df.columns[lower_cols.index(candidate)]
        break

# Regression targets
updrs_cols = [c for c in df.columns if 'updrs' in c.lower()]

# Group/patient column
group_col = None
for candidate in ['id','subject','subject#','patient_id','patient','name','file']:
    if candidate in lower_cols:
        group_col = df.columns[lower_cols.index(candidate)]
        break

print(f"Status column: {status_col}")
print(f"UPDRS columns: {updrs_cols}")
print(f"Group column: {group_col}")

has_classification = status_col is not None
has_regression = len(updrs_cols) > 0

# Feature columns
drop_candidates = []
if group_col: drop_candidates.append(group_col)
if status_col: drop_candidates.append(status_col)
drop_candidates += updrs_cols
for c in df.columns:
    if c.lower() in ['name','file']:
        drop_candidates.append(c)

feature_cols = [c for c in df.columns if c not in drop_candidates]
print(f"Feature columns: {len(feature_cols)}")

# ---------- 6) Enhanced Data Exploration ----------
print("\n" + "="*80)
print("6. ENHANCED DATA EXPLORATION")
print("="*80)

# Basic dataset info
dataset_info = {
    'Total Samples': len(df),
    'Total Features': len(feature_cols),
    'Parkinson Patients': df[status_col].sum() if has_classification else 'N/A',
    'Healthy Controls': len(df) - df[status_col].sum() if has_classification else 'N/A',
    'Unique Patients': df[group_col].nunique() if group_col else 'N/A'
}

print("Dataset Summary:")
for key, value in dataset_info.items():
    print(f"  {key}: {value}")

# Extract patient IDs if available
if group_col:
    df['patient_id'] = df[group_col].astype(str)
else:
    df['patient_id'] = df.index.astype(str)

# ---------- 7) Data Preprocessing Functions ----------
def preprocess_features(X):
    """Preprocess features: impute missing values and standardize"""
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    imp = SimpleImputer(strategy='median')
    X_num = pd.DataFrame(imp.fit_transform(X[num_cols]), columns=num_cols, index=X.index)

    cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        X_cat = pd.get_dummies(X[cat_cols].fillna('NA'), drop_first=True)
        X_proc = pd.concat([X_num, X_cat], axis=1)
    else:
        X_proc = X_num

    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_proc), columns=X_proc.columns, index=X_proc.index)
    return X_scaled, imp, scaler

# ---------- 8) Enhanced Visualization ----------
print("\n" + "="*80)
print("8. ENHANCED DATA VISUALIZATION")
print("="*80)

# 8.1 Class Distribution
if has_classification:
    plt.figure(figsize=(12, 10))

    plt.subplot(2, 2, 1)
    class_counts = df[status_col].value_counts()
    plt.pie(class_counts, labels=['Healthy', 'Parkinson\'s'], autopct='%1.1f%%',
            colors=['lightgreen', 'lightcoral'], startangle=90)
    plt.title('Class Distribution')

    # 8.2 Feature distributions
    plt.subplot(2, 2, 2)
    feature_to_plot = feature_cols[0] if feature_cols else df.columns[1]
    healthy_vals = df[df[status_col] == 0][feature_to_plot] if has_classification else df[feature_to_plot]
    parkinson_vals = df[df[status_col] == 1][feature_to_plot] if has_classification else None

    if parkinson_vals is not None:
        plt.hist([healthy_vals, parkinson_vals], bins=20, alpha=0.7,
                 label=['Healthy', 'Parkinson\'s'], color=['green', 'red'])
        plt.legend()
    else:
        plt.hist(healthy_vals, bins=20, alpha=0.7, color='blue')
    plt.xlabel(feature_to_plot)
    plt.ylabel('Frequency')
    plt.title(f'Distribution of {feature_to_plot}')

    # 8.3 Correlation Matrix (top 20 features)
    plt.subplot(2, 2, 3)
    numeric_cols = df[feature_cols].select_dtypes(include=[np.number]).columns
    top_features = numeric_cols[:min(20, len(numeric_cols))]
    corr_matrix = df[top_features].corr()

    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', center=0,
                square=True, linewidths=0.5, cbar_kws={"shrink": 0.8})
    plt.title('Feature Correlation Matrix (Top 20)')

    # 8.4 PCA Visualization
    plt.subplot(2, 2, 4)
    if has_classification:
        X_pca = df[feature_cols].select_dtypes(include=[np.number])
        X_pca = X_pca.fillna(X_pca.mean())
        scaler_pca = StandardScaler()
        X_pca_scaled = scaler_pca.fit_transform(X_pca)

        pca = PCA(n_components=2)
        X_pca_reduced = pca.fit_transform(X_pca_scaled)

        scatter = plt.scatter(X_pca_reduced[:, 0], X_pca_reduced[:, 1],
                             c=df[status_col], cmap='viridis', alpha=0.7)
        plt.colorbar(scatter)
        plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
        plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
        plt.title('PCA Visualization')

    plt.tight_layout()
    plt.savefig(f"{output_dirs['figures']}/data_exploration.png", bbox_inches='tight', dpi=300)
    plt.savefig(f"{output_dirs['figures']}/data_exploration.pdf", bbox_inches='tight')
    plt.close()

# ---------- 9) CLASSIFICATION PIPELINE ----------
results = {}
if has_classification:
    print("\n" + "="*80)
    print("9. CLASSIFICATION PIPELINE")
    print("="*80)

    y = df[status_col]
    X = df[feature_cols]

    # Preprocess features
    X_proc, imp_clf, scaler_clf = preprocess_features(X)

    # Feature selection
    if BORUTA_AVAILABLE:
        print("Attempting BorutaSHAP feature selection...")
        try:
            from boruta_shap import BorutaShap
            model_for_boruta = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
            boruta = BorutaShap(model=model_for_boruta, n_estimators='auto', sample=False, percentile=95, verbose=0)
            boruta.fit(X=X_proc, y=y, n_trials=50, random_state=42)
            selected = boruta.support
            sel_features = X_proc.columns[selected].tolist()
            if len(sel_features) == 0:
                raise Exception("Boruta selected 0 features")
        except Exception as e:
            print("BorutaSHAP failed, using SelectKBest:", e)
            BORUTA_AVAILABLE = False

    if not BORUTA_AVAILABLE:
        k = min(25, X_proc.shape[1])
        selector = SelectKBest(mutual_info_classif, k=k)
        selector.fit(X_proc.fillna(0), y)
        sel_features = X_proc.columns[selector.get_support()].tolist()

    print(f"Selected {len(sel_features)} features for classification")
    X_sel = X_proc[sel_features]

    # Patient-level splitting
    unique_patients = df['patient_id'].unique()
    patient_status = df.groupby('patient_id')[status_col].first()

    train_patients, temp_patients = train_test_split(
        unique_patients, test_size=0.3, random_state=42, stratify=patient_status
    )
    val_patients, test_patients = train_test_split(
        temp_patients, test_size=0.5, random_state=42,
        stratify=patient_status[patient_status.index.isin(temp_patients)]
    )

    train_mask = df['patient_id'].isin(train_patients)
    val_mask = df['patient_id'].isin(val_patients)
    test_mask = df['patient_id'].isin(test_patients)

    X_train, X_val, X_test = X_sel[train_mask], X_sel[val_mask], X_sel[test_mask]
    y_train, y_val, y_test = y[train_mask], y[val_mask], y[test_mask]

    print(f"Training set: {X_train.shape[0]} samples ({len(train_patients)} patients)")
    print(f"Validation set: {X_val.shape[0]} samples ({len(val_patients)} patients)")
    print(f"Test set: {X_test.shape[0]} samples ({len(test_patients)} patients)")

    # Handle class imbalance with SMOTE
    print(f"Before SMOTE - Class distribution: {pd.Series(y_train).value_counts().to_dict()}")
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    print(f"After SMOTE - Class distribution: {pd.Series(y_train_smote).value_counts().to_dict()}")

    # Define models
    clf_models = {
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
        'Support Vector Machine': SVC(probability=True, random_state=42),
        'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss', n_estimators=200),
        'LightGBM': lgb.LGBMClassifier(random_state=42, n_estimators=200),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=200)
    }

    # Train and evaluate models
    clf_results = {}
    for name, model in clf_models.items():
        print(f"Training {name}...")
        model.fit(X_train_smote, y_train_smote)

        y_pred = model.predict(X_test)
        y_pred_proba = model.predict_proba(X_test)[:, 1]

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        roc_auc = roc_auc_score(y_test, y_pred_proba)

        # Cross-validation
        cv_scores = cross_val_score(model, X_train_smote, y_train_smote, cv=5, scoring='accuracy')
        cv_mean = cv_scores.mean()
        cv_std = cv_scores.std()

        clf_results[name] = {
            'model': model,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'roc_auc': roc_auc,
            'cv_mean': cv_mean,
            'cv_std': cv_std,
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba
        }

        print(f"  Accuracy: {accuracy:.4f}, F1: {f1:.4f}, AUC: {roc_auc:.4f}")

    # Find best model
    best_clf_name = max(clf_results.keys(), key=lambda x: clf_results[x]['f1_score'])
    best_clf = clf_results[best_clf_name]['model']
    print(f"\nBest classifier: {best_clf_name} (F1: {clf_results[best_clf_name]['f1_score']:.4f})")

    # ---------- 9.1 Classification Visualizations ----------

    # Confusion Matrices
    fig, axes = plt.subplots(2, 3, figsize=(18, 12))
    axes = axes.ravel()

    for i, (name, results_dict) in enumerate(clf_results.items()):
        cm = confusion_matrix(y_test, results_dict['y_pred'])
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                    xticklabels=['Healthy', 'Parkinson\'s'],
                    yticklabels=['Healthy', 'Parkinson\'s'])
        axes[i].set_title(f'{name}\nAccuracy: {results_dict["accuracy"]:.3f}')
        axes[i].set_xlabel('Predicted')
        axes[i].set_ylabel('Actual')

    # Remove empty subplots
    for j in range(len(clf_models), 6):
        axes[j].axis('off')

    plt.tight_layout()
    plt.savefig(f"{output_dirs['figures']}/confusion_matrices.png", bbox_inches='tight', dpi=300)
    plt.savefig(f"{output_dirs['figures']}/confusion_matrices.pdf", bbox_inches='tight')
    plt.close()

    # ROC Curves
    plt.figure(figsize=(10, 8))
    for name, results_dict in clf_results.items():
        fpr, tpr, _ = roc_curve(y_test, results_dict['y_pred_proba'])
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.3f})')

    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.5)
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('ROC Curves - Classification')
    plt.legend(loc="lower right")
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig(f"{output_dirs['figures']}/roc_curves.png", bbox_inches='tight', dpi=300)
    plt.savefig(f"{output_dirs['figures']}/roc_curves.pdf", bbox_inches='tight')
    plt.close()

    # Performance Comparison
    metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
    metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']

    plt.figure(figsize=(15, 10))
    for i, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
        plt.subplot(2, 3, i+1)
        values = [clf_results[model][metric] for model in clf_models.keys()]
        bars = plt.barh(list(clf_models.keys()), values, color=plt.cm.Set3(np.linspace(0, 1, len(clf_models))))
        plt.xlabel(metric_name)
        plt.xlim(0, 1)
        for bar, value in zip(bars, values):
            plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                    f'{value:.3f}', ha='left', va='center', fontsize=8)
        plt.grid(True, alpha=0.3)

    plt.tight_layout()
    plt.savefig(f"{output_dirs['figures']}/model_performance.png", bbox_inches='tight', dpi=300)
    plt.savefig(f"{output_dirs['figures']}/model_performance.pdf", bbox_inches='tight')
    plt.close()

    results['classification'] = {
        'results': clf_results,
        'best_model': best_clf_name,
        'features': sel_features,
        'X_test': X_test,
        'y_test': y_test
    }

# ---------- 10) REGRESSION PIPELINE ----------
if has_regression:
    print("\n" + "="*80)
    print("10. REGRESSION PIPELINE")
    print("="*80)

    # Choose target
    target_reg = None
    for c in updrs_cols:
        if 'total' in c.lower() or 'motor' in c.lower():
            target_reg = c
            break
    if target_reg is None:
        target_reg = updrs_cols[0]

    print(f"Regression target: {target_reg}")

    y_reg = df[target_reg]
    X_reg = df[feature_cols]

    # Preprocess
    X_reg_proc, imp_reg, scaler_reg = preprocess_features(X_reg)

    # Feature selection
    k = min(25, X_reg_proc.shape[1])
    selector_reg = SelectKBest(mutual_info_regression, k=k)
    selector_reg.fit(X_reg_proc.fillna(0), y_reg.fillna(y_reg.median()))
    sel_features_reg = X_reg_proc.columns[selector_reg.get_support()].tolist()
    X_reg_sel = X_reg_proc[sel_features_reg]

    # Patient-level splitting
    unique_patients = df['patient_id'].unique()
    train_patients, temp_patients = train_test_split(unique_patients, test_size=0.3, random_state=42)
    val_patients, test_patients = train_test_split(temp_patients, test_size=0.5, random_state=42)

    train_mask = df['patient_id'].isin(train_patients)
    val_mask = df['patient_id'].isin(val_patients)
    test_mask = df['patient_id'].isin(test_patients)

    X_train_r, X_val_r, X_test_r = X_reg_sel[train_mask], X_reg_sel[val_mask], X_reg_sel[test_mask]
    y_train_r, y_val_r, y_test_r = y_reg[train_mask], y_reg[val_mask], y_reg[test_mask]

    # Regression models
    reg_models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(random_state=42),
        'Random Forest': RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
        'XGBoost': XGBRegressor(random_state=42, n_estimators=200),
        'LightGBM': lgb.LGBMRegressor(random_state=42, n_estimators=200),
        'Support Vector Regression': SVR()
    }

    reg_results = {}
    for name, model in reg_models.items():
        print(f"Training {name}...")
        model.fit(X_train_r, y_train_r)

        y_pred_r = model.predict(X_test_r)
        mae = mean_absolute_error(y_test_r, y_pred_r)
        rmse = mean_squared_error(y_test_r, y_pred_r, squared=False)
        r2 = r2_score(y_test_r, y_pred_r)

        reg_results[name] = {
            'model': model,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'y_pred': y_pred_r
        }

        print(f"  MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

    # Find best model
    best_reg_name = min(reg_results.keys(), key=lambda x: reg_results[x]['MAE'])
    best_reg = reg_results[best_reg_name]['model']
    print(f"\nBest regressor: {best_reg_name} (MAE: {reg_results[best_reg_name]['MAE']:.4f})")

    # Regression visualization
    plt.figure(figsize=(12, 5))

    plt.subplot(1, 2, 1)
    y_pred_best = reg_results[best_reg_name]['y_pred']
    plt.scatter(y_test_r, y_pred_best, alpha=0.6)
    plt.plot([y_test_r.min(), y_test_r.max()], [y_test_r.min(), y_test_r.max()], 'r--', lw=2)
    plt.xlabel('Actual')
    plt.ylabel('Predicted')
    plt.title(f'{best_reg_name} - Predicted vs Actual\nR² = {reg_results[best_reg_name]["R2"]:.3f}')

    plt.subplot(1, 2, 2)
    models = list(reg_results.keys())
    mae_scores = [reg_results[model]['MAE'] for model in models]
    plt.barh(models, mae_scores)
    plt.xlabel('MAE')
    plt.title('Model Comparison (MAE)')

    plt.tight_layout()
    plt.savefig(f"{output_dirs['figures']}/regression_results.png", bbox_inches='tight', dpi=300)
    plt.savefig(f"{output_dirs['figures']}/regression_results.pdf", bbox_inches='tight')
    plt.close()

    results['regression'] = {
        'results': reg_results,
        'best_model': best_reg_name,
        'features': sel_features_reg,
        'target': target_reg
    }

# ---------- 11) SHAP EXPLAINABILITY ----------
print("\n" + "="*80)
print("11. SHAP EXPLAINABILITY ANALYSIS")
print("="*80)

if SHAP_AVAILABLE:
    try:
        # Classification SHAP
        if 'classification' in results:
            best_clf_name = results['classification']['best_model']
            best_clf = results['classification']['results'][best_clf_name]['model']
            X_test_clf = results['classification']['X_test']

            if hasattr(best_clf, 'feature_importances_'):  # Tree-based model
                explainer = shap.TreeExplainer(best_clf)
                shap_values = explainer.shap_values(X_test_clf)

                # Summary plot
                plt.figure(figsize=(10, 8))
                shap.summary_plot(shap_values, X_test_clf, feature_names=X_test_clf.columns, show=False)
                plt.tight_layout()
                plt.savefig(f"{output_dirs['shap']}/shap_summary_classification.png", bbox_inches='tight', dpi=300)
                plt.savefig(f"{output_dirs['shap']}/shap_summary_classification.pdf", bbox_inches='tight')
                plt.close()

                # Bar plot
                plt.figure(figsize=(10, 8))
                shap.summary_plot(shap_values, X_test_clf, feature_names=X_test_clf.columns,
                                 plot_type="bar", show=False)
                plt.tight_layout()
                plt.savefig(f"{output_dirs['shap']}/shap_bar_classification.png", bbox_inches='tight', dpi=300)
                plt.savefig(f"{output_dirs['shap']}/shap_bar_classification.pdf", bbox_inches='tight')
                plt.close()

                print("Classification SHAP analysis completed")

        # Regression SHAP
        if 'regression' in results:
            best_reg_name = results['regression']['best_model']
            best_reg = results['regression']['results'][best_reg_name]['model']
            X_test_reg = X_test_r  # From regression pipeline

            if hasattr(best_reg, 'feature_importances_'):  # Tree-based model
                explainer_reg = shap.TreeExplainer(best_reg)
                shap_values_reg = explainer_reg.shap_values(X_test_reg)

                plt.figure(figsize=(10, 8))
                shap.summary_plot(shap_values_reg, X_test_reg, feature_names=X_test_reg.columns, show=False)
                plt.tight_layout()
                plt.savefig(f"{output_dirs['shap']}/shap_summary_regression.png", bbox_inches='tight', dpi=300)
                plt.savefig(f"{output_dirs['shap']}/shap_summary_regression.pdf", bbox_inches='tight')
                plt.close()

                print("Regression SHAP analysis completed")

    except Exception as e:
        print(f"SHAP analysis failed: {e}")
else:
    print("SHAP not available; skipping explainability analysis")

# ---------- 12) SAVE MODELS AND RESULTS ----------
print("\n" + "="*80)
print("12. SAVING MODELS AND RESULTS")
print("="*80)

# Save models
if 'classification' in results:
    best_clf = results['classification']['results'][results['classification']['best_model']]['model']
    joblib.dump(best_clf, f"{output_dirs['models']}/best_classifier.pkl")

if 'regression' in results:
    best_reg = results['regression']['results'][results['regression']['best_model']]['model']
    joblib.dump(best_reg, f"{output_dirs['models']}/best_regressor.pkl")

# Save results to CSV
summary_rows = []
if 'classification' in results:
    for name, res in results['classification']['results'].items():
        summary_rows.append({
            'Task': 'Classification',
            'Model': name,
            'Accuracy': res['accuracy'],
            'Precision': res['precision'],
            'Recall': res['recall'],
            'F1-Score': res['f1_score'],
            'ROC-AUC': res['roc_auc'],
            'CV_Accuracy_Mean': res['cv_mean'],
            'CV_Accuracy_Std': res['cv_std']
        })

if 'regression' in results:
    for name, res in results['regression']['results'].items():
        summary_rows.append({
            'Task': 'Regression',
            'Model': name,
            'MAE': res['MAE'],
            'RMSE': res['RMSE'],
            'R2': res['R2'],
            'Target': results['regression']['target']
        })

summary_df = pd.DataFrame(summary_rows)
summary_df.to_csv(f"{output_dirs['tables']}/model_results_summary.csv", index=False)
summary_df.to_excel(f"{output_dirs['tables']}/model_results_summary.xlsx", index=False)

# ---------- 13) GENERATE FINAL REPORT ----------
print("\n" + "="*80)
print("13. GENERATING FINAL REPORT")
print("="*80)

report_content = f"""
PARKINSON'S DISEASE DETECTION - COMPREHENSIVE ANALYSIS REPORT
================================================================================
Run Timestamp: {timestamp}
Output Directory: {output_base}

EXPERIMENT SUMMARY:
- Dataset: {os.path.basename(csv_path)}
- Total Samples: {dataset_info['Total Samples']}
- Features: {dataset_info['Total Features']}
- Parkinson's Patients: {dataset_info['Parkinson Patients']}
- Healthy Controls: {dataset_info['Healthy Controls']}
- Unique Patients: {dataset_info['Unique Patients']}

RESULTS:
"""

if 'classification' in results:
    best_clf = results['classification']['best_model']
    best_metrics = results['classification']['results'][best_clf]
    report_content += f"""
CLASSIFICATION RESULTS:
- Best Model: {best_clf}
- Test Accuracy: {best_metrics['accuracy']:.4f}
- Test F1-Score: {best_metrics['f1_score']:.4f}
- Test ROC-AUC: {best_metrics['roc_auc']:.4f}
- CV Accuracy: {best_metrics['cv_mean']:.4f} ± {best_metrics['cv_std']:.4f}

"""

if 'regression' in results:
    best_reg = results['regression']['best_model']
    best_metrics_reg = results['regression']['results'][best_reg]
    report_content += f"""
REGRESSION RESULTS:
- Target: {results['regression']['target']}
- Best Model: {best_reg}
- MAE: {best_metrics_reg['MAE']:.4f}
- RMSE: {best_metrics_reg['RMSE']:.4f}
- R²: {best_metrics_reg['R2']:.4f}

"""

report_content += f"""
FILES GENERATED:

FIGURES:
- data_exploration.png/pdf - Data distribution and correlations
- confusion_matrices.png/pdf - Confusion matrices for all models
- roc_curves.png/pdf - ROC curves for classification
- model_performance.png/pdf - Performance comparison
- regression_results.png/pdf - Regression results
- shap_*.png/pdf - SHAP explainability plots

TABLES:
- model_results_summary.csv/xlsx - Comprehensive results

MODELS:
- best_classifier.pkl - Best classification model
- best_regressor.pkl - Best regression model

All results are suitable for conference paper submission.
"""

with open(f"{output_dirs['results']}/analysis_report.txt", 'w') as f:
    f.write(report_content)

print(report_content)

# ---------- 14) FINAL SUMMARY ----------
print("\n" + "="*80)
print("EXPERIMENT COMPLETED SUCCESSFULLY!")
print("="*80)

print(f"\n📍 ALL OUTPUTS SAVED TO: {output_base}")
print(f"📅 Run ID: {timestamp}")

print(f"\n📊 FILES GENERATED:")
print("✅ /figures/ - All visualizations (PNG/PDF)")
print("✅ /tables/ - Results tables (CSV/XLSX)")
print("✅ /models/ - Trained models")
print("✅ /shap/ - Explainability analysis")
print("✅ /results/ - Comprehensive report")

print(f"\n🎯 READY FOR CONFERENCE PAPER SUBMISSION!")

ERROR: Could not find a version that satisfies the requirement boruta_shap (from versions: none)
ERROR: No matching distribution found for boruta_shap
Imports completed successfully
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_194448/figures
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_194448/tables
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_194448/models
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_194448/results
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_194448/shap

4. DATA LOADING AND SANITIZATION
Dataset shape: (195, 24)
Sanitized columns: ['name', 'MDVPFoHz', 'MDVPFhiHz', 'MDVPFloHz', 'MDVPJitter', 'MDVPJitterAbs', 'MDVPRAP', 'MDVPPPQ', 'JitterDDP', 'MDVPShimmer', 'MDVPShimmerdB', 'ShimmerAPQ3', 'ShimmerAPQ5', 'MDVPAPQ', 'ShimmerDDA', 'NHR', 'HN

In [22]:
# ---------- FULL PIPELINE (PNG-only, individual figures except ROC) ----------
# Run as a single Colab cell. Adjust paths as needed.

# ---------- 1) Environment setup ----------
!pip install -q pandas numpy scikit-learn matplotlib seaborn xgboost lightgbm shap imbalanced-learn joblib scipy plotly

# Optional BorutaSHAP
try:
    !pip install -q boruta_shap
    BORUTA_AVAILABLE = True
except Exception as e:
    BORUTA_AVAILABLE = False

# ---------- 2) Imports ----------
import os
import sys
import warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import joblib
import re
from datetime import datetime
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, mutual_info_classif, mutual_info_regression
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, confusion_matrix, classification_report,
                           mean_absolute_error, mean_squared_error, r2_score,
                           roc_curve, auc, precision_recall_curve, ConfusionMatrixDisplay)
from sklearn.decomposition import PCA
from xgboost import XGBClassifier, XGBRegressor
import lightgbm as lgb
from imblearn.over_sampling import SMOTE

# SHAP import with flag
try:
    import shap
    SHAP_AVAILABLE = True
except Exception:
    SHAP_AVAILABLE = False

warnings.filterwarnings("ignore")

# ---------- 3) Plot style & rcParams (publication friendly, PNG-only) ----------
plt.style.use('seaborn-v0_8-whitegrid')
plt.rcParams.update({
    'figure.dpi': 300,
    'savefig.dpi': 300,
    'font.size': 11,
    'axes.titlesize': 12,
    'axes.labelsize': 11,
    'legend.fontsize': 10,
    'xtick.labelsize': 9,
    'ytick.labelsize': 9,
    'lines.linewidth': 2.5
})
sns.set_palette("tab10")

print("Imports completed successfully")

# ---------- 4) Create Output Directory Structure ----------
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
output_base = f"/content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_{timestamp}"

output_dirs = {
    'figures': f"{output_base}/figures",
    'tables': f"{output_base}/tables",
    'models': f"{output_base}/models",
    'results': f"{output_base}/results",
    'shap': f"{output_base}/shap"
}

for dir_name, dir_path in output_dirs.items():
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created directory: {dir_path}")

# ---------- 5) Data Loading and Sanitization ----------
print("\n" + "="*80)
print("4. DATA LOADING AND SANITIZATION")
print("="*80)

# <-- EDIT THIS PATH if your CSV is elsewhere -->
csv_path = "/content/drive/MyDrive/ML COURSE PAPER/DATASET/Parkinsson disease.csv"
df = pd.read_csv(csv_path)
print(f"Dataset shape: {df.shape}")

# Sanitize column names
def sanitize_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'[^\w_]+', '', col)
        new_cols.append(new_col)
    df.columns = new_cols
    return df

df = sanitize_col_names(df)
print("Sanitized columns:", df.columns.tolist())

# ---------- 6) Auto-detect Columns ----------
lower_cols = [c.lower() for c in df.columns]

# Classification target
status_col = None
for candidate in ['status','class','target','parkinsons','label','diagnosis','is_parkinson','pd']:
    if candidate in lower_cols:
        status_col = df.columns[lower_cols.index(candidate)]
        break

# Regression targets (UPDRS)
updrs_cols = [c for c in df.columns if 'updrs' in c.lower()]

# Group/patient column
group_col = None
for candidate in ['id','subject','subject#','patient_id','patient','name','file']:
    if candidate in lower_cols:
        group_col = df.columns[lower_cols.index(candidate)]
        break

print(f"Status column: {status_col}")
print(f"UPDRS columns: {updrs_cols}")
print(f"Group column: {group_col}")

has_classification = status_col is not None
has_regression = len(updrs_cols) > 0

# Feature columns
drop_candidates = []
if group_col: drop_candidates.append(group_col)
if status_col: drop_candidates.append(status_col)
drop_candidates += updrs_cols
for c in df.columns:
    if c.lower() in ['name','file']:
        drop_candidates.append(c)

feature_cols = [c for c in df.columns if c not in drop_candidates]
print(f"Feature columns: {len(feature_cols)}")

# ---------- 7) Enhanced Data Exploration ----------
print("\n" + "="*80)
print("6. ENHANCED DATA EXPLORATION")
print("="*80)

# Basic dataset info
dataset_info = {
    'Total Samples': len(df),
    'Total Features': len(feature_cols),
    'Parkinson Patients': int(df[status_col].sum()) if has_classification else 'N/A',
    'Healthy Controls': int(len(df) - df[status_col].sum()) if has_classification else 'N/A',
    'Unique Patients': int(df[group_col].nunique()) if group_col else 'N/A'
}

print("Dataset Summary:")
for key, value in dataset_info.items():
    print(f"  {key}: {value}")

# Extract patient IDs if available
if group_col:
    df['patient_id'] = df[group_col].astype(str)
else:
    df['patient_id'] = df.index.astype(str)

# ---------- 8) Preprocessing helper ----------
def preprocess_features(X):
    """Preprocess features: impute missing values and standardize"""
    num_cols = X.select_dtypes(include=[np.number]).columns.tolist()
    imp = SimpleImputer(strategy='median')
    X_num = pd.DataFrame(imp.fit_transform(X[num_cols]), columns=num_cols, index=X.index)

    cat_cols = X.select_dtypes(include=['object','category']).columns.tolist()
    if cat_cols:
        X_cat = pd.get_dummies(X[cat_cols].fillna('NA'), drop_first=True)
        X_proc = pd.concat([X_num, X_cat], axis=1)
    else:
        X_proc = X_num

    scaler = StandardScaler()
    X_scaled = pd.DataFrame(scaler.fit_transform(X_proc), columns=X_proc.columns, index=X_proc.index)
    return X_scaled, imp, scaler

# ---------- 9) Data Exploration Plots (combined + individual PNGs) ----------
if has_classification:
    fig = plt.figure(figsize=(12, 10))
    ax1 = plt.subplot(2, 2, 1)
    class_counts = df[status_col].value_counts()
    ax1.pie(class_counts, labels=['Healthy', "Parkinson's"], autopct='%1.1f%%', startangle=90)
    ax1.set_title('Class Distribution')

    ax2 = plt.subplot(2, 2, 2)
    feature_to_plot = feature_cols[0] if feature_cols else df.columns[1]
    healthy_vals = df[df[status_col] == 0][feature_to_plot]
    parkinson_vals = df[df[status_col] == 1][feature_to_plot]
    ax2.hist([healthy_vals, parkinson_vals], bins=20, alpha=0.7, label=['Healthy', "Parkinson's"])
    ax2.legend()
    ax2.set_xlabel(feature_to_plot)
    ax2.set_ylabel('Frequency')
    ax2.set_title(f'Distribution of {feature_to_plot}')

    ax3 = plt.subplot(2, 2, 3)
    numeric_cols = df[feature_cols].select_dtypes(include=[np.number]).columns
    top_features = numeric_cols[:min(20, len(numeric_cols))]
    corr_matrix = df[top_features].corr()
    mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
    sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, ax=ax3)
    ax3.set_title('Feature Correlation Matrix (Top 20)')

    ax4 = plt.subplot(2, 2, 4)
    X_pca = df[feature_cols].select_dtypes(include=[np.number]).fillna(0)
    scaler_pca = StandardScaler()
    X_pca_scaled = scaler_pca.fit_transform(X_pca)
    pca = PCA(n_components=2)
    X_pca_reduced = pca.fit_transform(X_pca_scaled)
    scatter = ax4.scatter(X_pca_reduced[:, 0], X_pca_reduced[:, 1], c=df[status_col], cmap='viridis', alpha=0.7)
    plt.colorbar(scatter, ax=ax4)
    ax4.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
    ax4.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
    ax4.set_title('PCA Visualization')

    plt.tight_layout()
    data_expl_png = f"{output_dirs['figures']}/data_exploration.png"
    plt.savefig(data_expl_png, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Saved data exploration combined -> {data_expl_png}")

    # Individual subplots (recreate each cleanly)
    indiv_dir = f"{output_dirs['figures']}/data_exploration_individual"
    os.makedirs(indiv_dir, exist_ok=True)

    # 1: Class distribution
    fig, ax = plt.subplots(figsize=(6,5))
    ax.pie(class_counts, labels=['Healthy', "Parkinson's"], autopct='%1.1f%%', startangle=90)
    ax.set_title('Class Distribution')
    fn = f"{indiv_dir}/class_distribution.png"
    plt.tight_layout(); plt.savefig(fn, bbox_inches='tight', dpi=300); plt.close()
    print(f"Saved -> {fn}")

    # 2: Feature distribution
    fig, ax = plt.subplots(figsize=(6,5))
    ax.hist([healthy_vals, parkinson_vals], bins=20, alpha=0.7, label=['Healthy', "Parkinson's"])
    ax.legend(); ax.set_xlabel(feature_to_plot); ax.set_ylabel('Frequency'); ax.set_title(f'Distribution of {feature_to_plot}')
    fn = f"{indiv_dir}/feature_distribution_{feature_to_plot}.png"
    plt.tight_layout(); plt.savefig(fn, bbox_inches='tight', dpi=300); plt.close()
    print(f"Saved -> {fn}")

    # 3: Correlation
    fig, ax = plt.subplots(figsize=(8,6))
    sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', center=0, square=True, linewidths=0.5, cbar_kws={"shrink": 0.8}, ax=ax)
    ax.set_title('Feature Correlation Matrix (Top 20)')
    fn = f"{indiv_dir}/correlation_top20.png"
    plt.tight_layout(); plt.savefig(fn, bbox_inches='tight', dpi=300); plt.close()
    print(f"Saved -> {fn}")

    # 4: PCA
    fig, ax = plt.subplots(figsize=(6,5))
    ax.scatter(X_pca_reduced[:,0], X_pca_reduced[:,1], c=df[status_col], cmap='viridis', alpha=0.7)
    plt.colorbar(ax.collections[0], ax=ax)
    ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
    ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
    ax.set_title('PCA Visualization')
    fn = f"{indiv_dir}/pca_visualization.png"
    plt.tight_layout(); plt.savefig(fn, bbox_inches='tight', dpi=300); plt.close()
    print(f"Saved -> {fn}")

# ---------- 10) CLASSIFICATION PIPELINE ----------
results = {}
if has_classification:
    print("\n" + "="*80)
    print("9. CLASSIFICATION PIPELINE")
    print("="*80)

    y = df[status_col].astype(int)
    X = df[feature_cols]

    # Preprocess features
    X_proc, imp_clf, scaler_clf = preprocess_features(X)

    # Feature selection with BorutaSHAP if available, else SelectKBest
    sel_features = None
    if BORUTA_AVAILABLE:
        try:
            from boruta_shap import BorutaShap
            model_for_boruta = RandomForestClassifier(n_estimators=100, random_state=42, n_jobs=-1)
            boruta = BorutaShap(model=model_for_boruta, n_estimators='auto', sample=False, percentile=95, verbose=0)
            boruta.fit(X=X_proc, y=y, n_trials=50, random_state=42)
            selected = boruta.support
            sel_features = X_proc.columns[selected].tolist()
            if len(sel_features) == 0:
                sel_features = None
        except Exception as e:
            sel_features = None

    if not sel_features:
        k = min(25, X_proc.shape[1])
        selector = SelectKBest(mutual_info_classif, k=k)
        selector.fit(X_proc.fillna(0), y)
        sel_features = X_proc.columns[selector.get_support()].tolist()

    print(f"Selected {len(sel_features)} features for classification")
    X_sel = X_proc[sel_features]

    # Patient-level splitting with proper stratify alignment
    unique_patients = df['patient_id'].unique()
    patient_status = df.groupby('patient_id')[status_col].first()
    stratify_labels = patient_status.loc[unique_patients].values  # aligned labels

    train_patients, temp_patients = train_test_split(
        unique_patients, test_size=0.3, random_state=42, stratify=stratify_labels
    )
    temp_stratify = patient_status.loc[temp_patients].values
    val_patients, test_patients = train_test_split(
        temp_patients, test_size=0.5, random_state=42, stratify=temp_stratify
    )

    train_mask = df['patient_id'].isin(train_patients)
    val_mask = df['patient_id'].isin(val_patients)
    test_mask = df['patient_id'].isin(test_patients)

    X_train, X_val, X_test = X_sel[train_mask], X_sel[val_mask], X_sel[test_mask]
    y_train, y_val, y_test = y[train_mask], y[val_mask], y[test_mask]

    print(f"Training set: {X_train.shape[0]} samples ({len(train_patients)} patients)")
    print(f"Validation set: {X_val.shape[0]} samples ({len(val_patients)} patients)")
    print(f"Test set: {X_test.shape[0]} samples ({len(test_patients)} patients)")

    # Handle class imbalance with SMOTE
    print(f"Before SMOTE - Class distribution: {pd.Series(y_train).value_counts().to_dict()}")
    smote = SMOTE(random_state=42)
    X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)
    print(f"After SMOTE - Class distribution: {pd.Series(y_train_smote).value_counts().to_dict()}")

    # Define models
    clf_models = {
        'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
        'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
        'Support Vector Machine': SVC(probability=True, random_state=42),
        'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss', n_estimators=200),
        'LightGBM': lgb.LGBMClassifier(random_state=42, n_estimators=200),
        'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=200)
    }

    # Train and evaluate models
    clf_results = {}
    for name, model in clf_models.items():
        print(f"Training {name}...")
        model.fit(X_train_smote, y_train_smote)

        y_pred = model.predict(X_test)
        # Safely get predict_proba (fall back if not available)
        if hasattr(model, "predict_proba"):
            y_pred_proba = model.predict_proba(X_test)[:, 1]
        elif hasattr(model, "decision_function"):
            # scale decision_function to [0,1] with min-max as fallback for ROC calculations
            scores = model.decision_function(X_test)
            y_pred_proba = (scores - scores.min()) / (scores.max() - scores.min() + 1e-12)
        else:
            y_pred_proba = y_pred  # fallback (discrete)

        # Calculate metrics
        accuracy = accuracy_score(y_test, y_pred)
        precision = precision_score(y_test, y_pred, zero_division=0)
        recall = recall_score(y_test, y_pred, zero_division=0)
        f1 = f1_score(y_test, y_pred, zero_division=0)
        try:
            roc_auc = roc_auc_score(y_test, y_pred_proba)
        except Exception:
            roc_auc = float('nan')

        # Cross-validation (on resampled train)
        try:
            cv_scores = cross_val_score(model, X_train_smote, y_train_smote, cv=5, scoring='accuracy')
            cv_mean = float(cv_scores.mean())
            cv_std = float(cv_scores.std())
        except Exception:
            cv_mean, cv_std = float('nan'), float('nan')

        clf_results[name] = {
            'model': model,
            'accuracy': accuracy,
            'precision': precision,
            'recall': recall,
            'f1_score': f1,
            'roc_auc': roc_auc,
            'cv_mean': cv_mean,
            'cv_std': cv_std,
            'y_pred': y_pred,
            'y_pred_proba': y_pred_proba
        }

        print(f"  Accuracy: {accuracy:.4f}, F1: {f1:.4f}, AUC: {roc_auc:.4f}")

    # Find best model by F1
    best_clf_name = max(clf_results.keys(), key=lambda x: clf_results[x]['f1_score'])
    best_clf = clf_results[best_clf_name]['model']
    print(f"\nBest classifier: {best_clf_name} (F1: {clf_results[best_clf_name]['f1_score']:.4f})")

    # Save classification results to results dict
    results['classification'] = {
        'results': clf_results,
        'best_model': best_clf_name,
        'features': sel_features,
        'X_test': X_test,
        'y_test': y_test
    }

# ---------- 11) Visualization SAVES (PNG-only) ----------
if has_classification:
    # Confusion matrices: one PNG per model
    indiv_cm_dir = f"{output_dirs['figures']}/confusion_matrices_individual"
    os.makedirs(indiv_cm_dir, exist_ok=True)
    cm_labels = ['Healthy', "Parkinson's"]

    for name, res in clf_results.items():
        cm = confusion_matrix(y_test, res['y_pred'])
        cm_sum = cm.sum(axis=1)[:, None]
        cm_pct = (cm / cm_sum.astype(float)) * 100.0
        fig, ax = plt.subplots(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax, xticklabels=cm_labels, yticklabels=cm_labels, cbar=False)
        ax.set_title(f'{name}\nAcc: {res["accuracy"]:.3f}  F1: {res["f1_score"]:.3f}')
        ax.set_xlabel('Predicted'); ax.set_ylabel('Actual')
        for i in range(cm.shape[0]):
            for j in range(cm.shape[1]):
                ax.text(j + 0.05, i + 0.6, f'{cm_pct[i,j]:.1f}%', fontsize=9, color='black')
        fn_png = f"{indiv_cm_dir}/{name.replace(' ', '_')}_confusion_matrix.png"
        plt.tight_layout(); plt.savefig(fn_png, bbox_inches='tight', dpi=300); plt.close()
        print(f"Saved confusion matrix (PNG only) -> {fn_png}")

    # Combined confusion matrix grid (optional combined)
    n_models = len(clf_models)
    cols = min(3, n_models)
    rows = (n_models + cols - 1) // cols
    fig, axes = plt.subplots(rows, cols, figsize=(6*cols, 4*rows))
    axes = axes.ravel()
    for i, (name, res) in enumerate(clf_results.items()):
        cm = confusion_matrix(y_test, res['y_pred'])
        sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i], xticklabels=cm_labels, yticklabels=cm_labels, cbar=False)
        axes[i].set_title(f'{name}\nAcc: {res["accuracy"]:.3f}  F1: {res["f1_score"]:.3f}')
        axes[i].set_xlabel('Predicted'); axes[i].set_ylabel('Actual')
    for j in range(n_models, len(axes)):
        axes[j].axis('off')
    combined_cm_png = f"{output_dirs['figures']}/confusion_matrices_combined.png"
    plt.tight_layout(); plt.savefig(combined_cm_png, bbox_inches='tight', dpi=300); plt.close()
    print(f"Saved combined confusion matrices (PNG only) -> {combined_cm_png}")

    # Model performance metrics: combined + individual PNGs for each metric
    metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
    metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']
    perf_dir = f"{output_dirs['figures']}/model_performance_individual"
    os.makedirs(perf_dir, exist_ok=True)

    # Combined figure
    fig = plt.figure(figsize=(15, 10))
    for i, (metric, metric_name) in enumerate(zip(metrics, metric_names), start=1):
        ax = plt.subplot(2, 3, i)
        values = [clf_results[m][metric] for m in clf_models.keys()]
        bars = ax.barh(list(clf_models.keys()), values)
        ax.set_xlabel(metric_name)
        ax.set_xlim(0, 1)
        for bar, value in zip(bars, values):
            ax.text(value + 0.01, bar.get_y() + bar.get_height()/2, f'{value:.3f}', va='center', fontsize=8)
        ax.grid(alpha=0.3)
    combined_perf_png = f"{output_dirs['figures']}/model_performance.png"
    plt.tight_layout(); plt.savefig(combined_perf_png, bbox_inches='tight', dpi=300); plt.close()
    print(f"Saved combined performance figure -> {combined_perf_png}")

    # Individual metric plots
    for metric, metric_name in zip(metrics, metric_names):
        fig, ax = plt.subplots(figsize=(8, max(3, len(clf_models)*0.5)))
        values = [clf_results[m][metric] for m in clf_models.keys()]
        bars = ax.barh(list(clf_models.keys()), values)
        ax.set_xlabel(metric_name)
        ax.set_xlim(0, 1)
        for bar, value in zip(bars, values):
            ax.text(value + 0.01, bar.get_y() + bar.get_height()/2, f'{value:.3f}', va='center', fontsize=8)
        ax.grid(alpha=0.3)
        fn = f"{perf_dir}/{metric_name.replace(' ', '_')}.png"
        plt.tight_layout(); plt.savefig(fn, bbox_inches='tight', dpi=300); plt.close()
        print(f"Saved metric plot -> {fn}")

# ---------- 12) REGRESSION PIPELINE ----------
if has_regression:
    print("\n" + "="*80)
    print("10. REGRESSION PIPELINE")
    print("="*80)

    # Choose target
    target_reg = None
    for c in updrs_cols:
        if 'total' in c.lower() or 'motor' in c.lower():
            target_reg = c
            break
    if target_reg is None:
        target_reg = updrs_cols[0]

    print(f"Regression target: {target_reg}")

    y_reg = df[target_reg]
    X_reg = df[feature_cols]

    X_reg_proc, imp_reg, scaler_reg = preprocess_features(X_reg)

    k = min(25, X_reg_proc.shape[1])
    selector_reg = SelectKBest(mutual_info_regression, k=k)
    selector_reg.fit(X_reg_proc.fillna(0), y_reg.fillna(y_reg.median()))
    sel_features_reg = X_reg_proc.columns[selector_reg.get_support()].tolist()
    X_reg_sel = X_reg_proc[sel_features_reg]

    unique_patients = df['patient_id'].unique()
    train_patients, temp_patients = train_test_split(unique_patients, test_size=0.3, random_state=42)
    val_patients, test_patients = train_test_split(temp_patients, test_size=0.5, random_state=42)

    train_mask = df['patient_id'].isin(train_patients)
    val_mask = df['patient_id'].isin(val_patients)
    test_mask = df['patient_id'].isin(test_patients)

    X_train_r, X_val_r, X_test_r = X_reg_sel[train_mask], X_reg_sel[val_mask], X_reg_sel[test_mask]
    y_train_r, y_val_r, y_test_r = y_reg[train_mask], y_reg[val_mask], y_reg[test_mask]

    # Regression models
    reg_models = {
        'Linear Regression': LinearRegression(),
        'Ridge Regression': Ridge(random_state=42),
        'Random Forest': RandomForestRegressor(n_estimators=200, random_state=42, n_jobs=-1),
        'XGBoost': XGBRegressor(random_state=42, n_estimators=200),
        'LightGBM': lgb.LGBMRegressor(random_state=42, n_estimators=200),
        'Support Vector Regression': SVR()
    }

    reg_results = {}
    for name, model in reg_models.items():
        print(f"Training {name}...")
        model.fit(X_train_r, y_train_r)

        y_pred_r = model.predict(X_test_r)
        mae = mean_absolute_error(y_test_r, y_pred_r)
        rmse = mean_squared_error(y_test_r, y_pred_r, squared=False)
        r2 = r2_score(y_test_r, y_pred_r)

        reg_results[name] = {
            'model': model,
            'MAE': mae,
            'RMSE': rmse,
            'R2': r2,
            'y_pred': y_pred_r
        }

        print(f"  MAE: {mae:.4f}, RMSE: {rmse:.4f}, R2: {r2:.4f}")

    best_reg_name = min(reg_results.keys(), key=lambda x: reg_results[x]['MAE'])
    best_reg = reg_results[best_reg_name]['model']
    print(f"\nBest regressor: {best_reg_name} (MAE: {reg_results[best_reg_name]['MAE']:.4f})")

    # Regression visualizations (PNG-only)
    fig = plt.figure(figsize=(12, 5))
    ax1 = plt.subplot(1, 2, 1)
    y_pred_best = reg_results[best_reg_name]['y_pred']
    ax1.scatter(y_test_r, y_pred_best, alpha=0.6)
    ax1.plot([y_test_r.min(), y_test_r.max()], [y_test_r.min(), y_test_r.max()], 'r--', lw=2)
    ax1.set_xlabel('Actual'); ax1.set_ylabel('Predicted')
    ax1.set_title(f'{best_reg_name} - Predicted vs Actual\nR² = {reg_results[best_reg_name]["R2"]:.3f}')

    ax2 = plt.subplot(1, 2, 2)
    models = list(reg_results.keys())
    mae_scores = [reg_results[m]['MAE'] for m in models]
    ax2.barh(models, mae_scores)
    ax2.set_xlabel('MAE')
    ax2.set_title('Model Comparison (MAE)')

    plt.tight_layout()
    reg_png = f"{output_dirs['figures']}/regression_results.png"
    plt.savefig(reg_png, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Saved regression results -> {reg_png}")

    results['regression'] = {
        'results': reg_results,
        'best_model': best_reg_name,
        'features': sel_features_reg,
        'target': target_reg
    }

# ---------- 13) SHAP EXPLAINABILITY (PNG-only) ----------
print("\n" + "="*80)
print("11. SHAP EXPLAINABILITY ANALYSIS")
print("="*80)

if SHAP_AVAILABLE:
    try:
        if 'classification' in results:
            best_clf_name = results['classification']['best_model']
            best_clf = results['classification']['results'][best_clf_name]['model']
            X_test_clf = results['classification']['X_test']
            if hasattr(best_clf, 'feature_importances_') or hasattr(best_clf, 'predict_proba'):
                explainer = shap.TreeExplainer(best_clf) if hasattr(best_clf, 'predict_proba') or hasattr(best_clf, 'feature_importances_') else shap.Explainer(best_clf, X_test_clf)
                shap_values = explainer.shap_values(X_test_clf) if hasattr(explainer, 'shap_values') else explainer(X_test_clf)

                # SHAP summary
                plt.figure(figsize=(10, 8))
                try:
                    shap.summary_plot(shap_values, X_test_clf, feature_names=X_test_clf.columns, show=False)
                    shap_summary_png = f"{output_dirs['shap']}/shap_summary_classification.png"
                    plt.tight_layout(); plt.savefig(shap_summary_png, bbox_inches='tight', dpi=300); plt.close()
                    print(f"Saved SHAP summary -> {shap_summary_png}")
                except Exception as e:
                    print(f"SHAP summary plotting failed: {e}")

                # SHAP bar
                plt.figure(figsize=(10, 8))
                try:
                    shap.summary_plot(shap_values, X_test_clf, feature_names=X_test_clf.columns, plot_type="bar", show=False)
                    shap_bar_png = f"{output_dirs['shap']}/shap_bar_classification.png"
                    plt.tight_layout(); plt.savefig(shap_bar_png, bbox_inches='tight', dpi=300); plt.close()
                    print(f"Saved SHAP bar -> {shap_bar_png}")
                except Exception as e:
                    print(f"SHAP bar plotting failed: {e}")
    except Exception as e:
        print(f"SHAP analysis failed: {e}")
else:
    print("SHAP not available; skipping explainability analysis")

# ---------- 14) SAVE MODELS AND RESULTS (and tables) ----------
print("\n" + "="*80)
print("12. SAVING MODELS AND RESULTS")
print("="*80)

# Save best models
if 'classification' in results:
    best_clf = results['classification']['results'][results['classification']['best_model']]['model']
    joblib.dump(best_clf, f"{output_dirs['models']}/best_classifier.pkl")
    print(f"Saved best classifier -> {output_dirs['models']}/best_classifier.pkl")

if 'regression' in results:
    best_reg = results['regression']['results'][results['regression']['best_model']]['model']
    joblib.dump(best_reg, f"{output_dirs['models']}/best_regressor.pkl")
    print(f"Saved best regressor -> {output_dirs['models']}/best_regressor.pkl")

# Save summary table
summary_rows = []
if 'classification' in results:
    for name, res in results['classification']['results'].items():
        summary_rows.append({
            'Task': 'Classification',
            'Model': name,
            'Accuracy': res['accuracy'],
            'Precision': res['precision'],
            'Recall': res['recall'],
            'F1-Score': res['f1_score'],
            'ROC-AUC': res['roc_auc'],
            'CV_Accuracy_Mean': res['cv_mean'],
            'CV_Accuracy_Std': res['cv_std']
        })

if 'regression' in results:
    for name, res in results['regression']['results'].items():
        summary_rows.append({
            'Task': 'Regression',
            'Model': name,
            'MAE': res['MAE'],
            'RMSE': res['RMSE'],
            'R2': res['R2'],
            'Target': results['regression']['target']
        })

summary_df = pd.DataFrame(summary_rows)
summary_csv = f"{output_dirs['tables']}/model_results_summary.csv"
summary_xlsx = f"{output_dirs['tables']}/model_results_summary.xlsx"
summary_df.to_csv(summary_csv, index=False)
summary_df.to_excel(summary_xlsx, index=False)
print(f"Saved results summary -> {summary_csv} and {summary_xlsx}")

# ---------- 15) FINAL REPORT (text) ----------
report_content = f"""
PARKINSON'S DISEASE DETECTION - COMPREHENSIVE ANALYSIS REPORT
================================================================================
Run Timestamp: {timestamp}
Output Directory: {output_base}

EXPERIMENT SUMMARY:
- Dataset: {os.path.basename(csv_path)}
- Total Samples: {dataset_info['Total Samples']}
- Features: {dataset_info['Total Features']}
- Parkinson's Patients: {dataset_info['Parkinson Patients']}
- Healthy Controls: {dataset_info['Healthy Controls']}
- Unique Patients: {dataset_info['Unique Patients']}

RESULTS:
"""

if 'classification' in results:
    best_clf = results['classification']['best_model']
    best_metrics = results['classification']['results'][best_clf]
    report_content += f"""
CLASSIFICATION RESULTS:
- Best Model: {best_clf}
- Test Accuracy: {best_metrics['accuracy']:.4f}
- Test F1-Score: {best_metrics['f1_score']:.4f}
- Test ROC-AUC: {best_metrics['roc_auc']:.4f}
- CV Accuracy: {best_metrics['cv_mean']:.4f} ± {best_metrics['cv_std']:.4f}

"""

if 'regression' in results:
    best_reg = results['regression']['best_model']
    best_metrics_reg = results['regression']['results'][best_reg]
    report_content += f"""
REGRESSION RESULTS:
- Target: {results['regression']['target']}
- Best Model: {best_reg}
- MAE: {best_metrics_reg['MAE']:.4f}
- RMSE: {best_metrics_reg['RMSE']:.4f}
- R²: {best_metrics_reg['R2']:.4f}

"""

report_content += f"""
FILES GENERATED (PNG only):

FIGURES:
- data_exploration.png - Data distribution and correlations
- data_exploration_individual/* - Individual exploration PNGs
- confusion_matrices_individual/* - One PNG per model confusion matrix
- confusion_matrices_combined.png - Combined grid (optional)
- model_performance.png and model_performance_individual/* - Performance comparisons
- roc_curves_polished.png - ROC curves (combined)
- regression_results.png - Regression results
- shap/*.png - SHAP explainability plots (if available)

TABLES:
- model_results_summary.csv/xlsx

MODELS:
- best_classifier.pkl - Best classification model
- best_regressor.pkl - Best regression model (if present)

All results are saved under: {output_base}
"""

with open(f"{output_dirs['results']}/analysis_report.txt", 'w') as f:
    f.write(report_content)

print(report_content)

# ---------- 16) ROC CURVE (polished single combined PNG) ----------
if has_classification:
    plt.figure(figsize=(10, 8))
    sns.set_style("whitegrid")
    colors = sns.color_palette("tab10", n_colors=len(clf_results))
    for (name, res), c in zip(clf_results.items(), colors):
        y_score = res.get('y_pred_proba', None)
        if y_score is None:
            if hasattr(res['model'], "decision_function"):
                y_score = res['model'].decision_function(X_test)
            else:
                y_score = res['y_pred']
        fpr, tpr, _ = roc_curve(y_test, y_score)
        roc_auc = auc(fpr, tpr)
        plt.plot(fpr, tpr, label=f"{name} (AUC = {roc_auc:.3f})", linewidth=3.0, alpha=0.9)

    plt.plot([0, 1], [0, 1], linestyle='--', linewidth=1.5, alpha=0.6, color='gray')
    plt.xlim([-0.01, 1.01]); plt.ylim([-0.01, 1.01])
    plt.xlabel('False Positive Rate', fontsize=12); plt.ylabel('True Positive Rate', fontsize=12)
    plt.title('ROC Curves - Classification (Test Set)', fontsize=13)
    plt.legend(loc='center left', bbox_to_anchor=(1.02, 0.5))
    plt.grid(alpha=0.3, linestyle='--')
    roc_png = f"{output_dirs['figures']}/roc_curves_polished.png"
    plt.tight_layout(rect=[0, 0, 0.78, 1])
    plt.savefig(roc_png, bbox_inches='tight', dpi=300)
    plt.close()
    print(f"Saved ROC curves (PNG only) -> {roc_png}")

print("\n" + "="*80)
print("EXPERIMENT COMPLETED SUCCESSFULLY!")
print("="*80)
print(f"All outputs saved to: {output_base}")


ERROR: Could not find a version that satisfies the requirement boruta_shap (from versions: none)
ERROR: No matching distribution found for boruta_shap
Imports completed successfully
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_195510/figures
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_195510/tables
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_195510/models
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_195510/results
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/Run_20250926_195510/shap

4. DATA LOADING AND SANITIZATION
Dataset shape: (195, 24)
Sanitized columns: ['name', 'MDVPFoHz', 'MDVPFhiHz', 'MDVPFloHz', 'MDVPJitter', 'MDVPJitterAbs', 'MDVPRAP', 'MDVPPPQ', 'JitterDDP', 'MDVPShimmer', 'MDVPShimmerdB', 'ShimmerAPQ3', 'ShimmerAPQ5', 'MDVPAPQ', 'ShimmerDDA', 'NHR', 'HN

In [25]:
# =============================================================================
# COMPREHENSIVE PARKINSON'S DISEASE DETECTION PIPELINE WITH ENHANCED VISUALIZATIONS
# =============================================================================

# ---------- 0) Environment Setup ----------
!pip install -q pandas numpy scikit-learn matplotlib seaborn xgboost lightgbm shap imbalanced-learn joblib scipy plotly
try:
    !pip install -q boruta_shap
    BORUTA_SHAP_INSTALLED = True
except Exception:
    BORUTA_SHAP_INSTALLED = False

# ---------- 1) Imports ----------
import os
import sys
import warnings
import math
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from plotly.subplots import make_subplots
import joblib
import re
from sklearn.model_selection import train_test_split, cross_val_score, StratifiedKFold
from sklearn.preprocessing import StandardScaler, LabelEncoder
from sklearn.impute import SimpleImputer
from sklearn.feature_selection import SelectKBest, mutual_info_classif, RFE
from sklearn.ensemble import RandomForestClassifier, RandomForestRegressor, GradientBoostingClassifier
from sklearn.svm import SVC, SVR
from sklearn.linear_model import LogisticRegression, LinearRegression, Ridge
from sklearn.metrics import (accuracy_score, precision_score, recall_score, f1_score,
                           roc_auc_score, confusion_matrix, classification_report,
                           mean_absolute_error, mean_squared_error, r2_score,
                           roc_curve, auc, precision_recall_curve)
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE
from xgboost import XGBClassifier, XGBRegressor
import lightgbm as lgb
from imblearn.over_sampling import SMOTE
import shap

warnings.filterwarnings("ignore")

# Set style for publication-quality figures
plt.style.use('seaborn-v0_8-whitegrid')
sns.set_palette("husl")
plt.rcParams['figure.dpi'] = 300
plt.rcParams['savefig.dpi'] = 300
plt.rcParams['font.size'] = 10

print("Imports completed successfully")
print(f"SHAP version: {shap.__version__}")

# Define the Google Drive output directory
output_base_dir = '/content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/'

# ---------- 2) Create Output Directory Structure ----------
output_dirs = [os.path.join(output_base_dir, 'figures'),
               os.path.join(output_base_dir, 'tables'),
               os.path.join(output_base_dir, 'models'),
               os.path.join(output_base_dir, 'results')]
for dir_path in output_dirs:
    os.makedirs(dir_path, exist_ok=True)
    print(f"Created directory: {dir_path}")

# ---------- 3) Data Loading and Exploration ----------
print("\n" + "="*80)
print("3. DATA LOADING AND EXPLORATION")
print("="*80)

# Load dataset
df = pd.read_csv('/content/drive/MyDrive/ML COURSE PAPER/DATASET/Parkinsson disease.csv')
print(f"Dataset shape: {df.shape}")

# Sanitize column names for LightGBM compatibility
def sanitize_col_names(df):
    cols = df.columns
    new_cols = []
    for col in cols:
        new_col = re.sub(r'[^\w_]+', '', col) # Remove special characters
        new_cols.append(new_col)
    df.columns = new_cols
    return df

df = sanitize_col_names(df)
print("Columns after sanitizing:", df.columns.tolist())


# Basic dataset info
dataset_info = {
    'Total Samples': len(df),
    'Total Features': len(df.columns) - 2,  # excluding name and status
    'Parkinson Patients': df['status'].sum(),
    'Healthy Controls': len(df) - df['status'].sum(),
    'Parkinson Percentage': f"{df['status'].sum()/len(df)*100:.1f}%",
    'Healthy Percentage': f"{(len(df) - df['status'].sum())/len(df)*100:.1f}%"
}

print("\nDataset Summary:")
for key, value in dataset_info.items():
    print(f"{key}: {value}")

# Extract patient IDs
df['patient_id'] = df['name'].str.extract(r'(phon_R\d+_S\d+)')
print(f"Unique patients: {df['patient_id'].nunique()}")

# ---------- 4) Enhanced Data Visualization ----------
print("\n" + "="*80)
print("4. ENHANCED DATA VISUALIZATION")
print("="*80)

# 4.1 Class Distribution Pie Chart
plt.figure(figsize=(10, 6))
plt.subplot(1, 2, 1)
class_counts = df['status'].value_counts()
plt.pie(class_counts, labels=['Healthy', 'Parkinson\'s'], autopct='%1.1f%%',
        colors=['lightgreen', 'lightcoral'], startangle=90)
plt.title('Class Distribution')

# 4.2 Feature Distribution by Class
plt.subplot(1, 2, 2)
# Select a representative feature for visualization
feature_to_plot = 'MDVPFoHz' if 'MDVPFoHz' in df.columns else df.columns[1]
if feature_to_plot in df.columns:
    healthy_vals = df[df['status'] == 0][feature_to_plot]
    parkinson_vals = df[df['status'] == 1][feature_to_plot]
    plt.hist([healthy_vals, parkinson_vals], bins=20, alpha=0.7,
             label=['Healthy', 'Parkinson\'s'], color=['green', 'red'])
    plt.xlabel(feature_to_plot)
    plt.ylabel('Frequency')
    plt.legend()
    plt.title(f'Distribution of {feature_to_plot}')

plt.tight_layout()
plt.savefig(os.path.join(output_base_dir, 'figures/class_distribution.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(output_base_dir, 'figures/class_distribution.pdf'), bbox_inches='tight')
plt.close()

# 4.3 Correlation Matrix Heatmap
plt.figure(figsize=(16, 14))
numeric_cols = df.select_dtypes(include=[np.number]).columns
corr_matrix = df[numeric_cols].corr()

# Create mask for upper triangle
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))

sns.heatmap(corr_matrix, mask=mask, cmap='coolwarm', center=0,
            square=True, linewidths=0.5, cbar_kws={"shrink": 0.8},
            annot=False, fmt=".2f")
plt.title('Feature Correlation Matrix', fontsize=14, pad=20)
plt.tight_layout()
plt.savefig(os.path.join(output_base_dir, 'figures/correlation_matrix.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(output_base_dir, 'figures/correlation_matrix.pdf'), bbox_inches='tight')
plt.close()

# 4.4 PCA Visualization
plt.figure(figsize=(12, 5))

# Prepare data for PCA
X_pca = df.drop(['name', 'status', 'patient_id'], axis=1, errors='ignore')
X_pca = X_pca.select_dtypes(include=[np.number])
X_pca = X_pca.fillna(X_pca.mean())

# Standardize
scaler_pca = StandardScaler()
X_pca_scaled = scaler_pca.fit_transform(X_pca)

# Apply PCA
pca = PCA(n_components=2)
X_pca_reduced = pca.fit_transform(X_pca_scaled)

plt.subplot(1, 2, 1)
scatter = plt.scatter(X_pca_reduced[:, 0], X_pca_reduced[:, 1],
                     c=df['status'], cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%} variance)')
plt.ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%} variance)')
plt.title('PCA - Parkinson vs Healthy')

# 4.5 t-SNE Visualization
plt.subplot(1, 2, 2)
tsne = TSNE(n_components=2, random_state=42, perplexity=30)
X_tsne = tsne.fit_transform(X_pca_scaled)

scatter = plt.scatter(X_tsne[:, 0], X_tsne[:, 1],
                     c=df['status'], cmap='viridis', alpha=0.7)
plt.colorbar(scatter)
plt.xlabel('t-SNE 1')
plt.ylabel('t-SNE 2')
plt.title('t-SNE Visualization')

plt.tight_layout()
plt.savefig(os.path.join(output_base_dir, 'figures/dimensionality_reduction.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(output_base_dir, 'figures/dimensionality_reduction.pdf'), bbox_inches='tight')
plt.close()

print("Enhanced data visualizations saved")

# ---------- 5) Data Preprocessing ----------
print("\n" + "="*80)
print("5. DATA PREPROCESSING")
print("="*80)

# Prepare features and target
X = df.drop(['name', 'status', 'patient_id'], axis=1, errors='ignore')
y = df['status']

# Handle missing values
imputer = SimpleImputer(strategy='median')
X_imputed = pd.DataFrame(imputer.fit_transform(X), columns=X.columns)

# Standardize features
scaler = StandardScaler()
X_scaled = pd.DataFrame(scaler.fit_transform(X_imputed), columns=X.columns)

print("Data preprocessing completed")
print(f"Final feature set: {X_scaled.shape[1]} features")

# ---------- 6) Feature Selection ----------
print("\n" + "="*80)
print("6. FEATURE SELECTION")
print("="*80)

# Select top features using mutual information
selector = SelectKBest(mutual_info_classif, k=min(20, X_scaled.shape[1]))
X_selected = selector.fit_transform(X_scaled, y)
selected_features = X_scaled.columns[selector.get_support()].tolist()

print(f"Selected {len(selected_features)} features:")
for i, feature in enumerate(selected_features, 1):
    print(f"{i:2d}. {feature}")

# Save feature selection results
feature_importance_df = pd.DataFrame({
    'Feature': X_scaled.columns,
    'Importance': selector.scores_,
    'Selected': selector.get_support()
}).sort_values('Importance', ascending=False)

feature_importance_df.to_csv(os.path.join(output_base_dir, 'tables/feature_importance.csv'), index=False)

# Visualize feature importance
plt.figure(figsize=(12, 8))
top_features = feature_importance_df.head(15)
sns.barplot(data=top_features, x='Importance', y='Feature')
plt.title('Top 15 Most Important Features (Mutual Information)')
plt.tight_layout()
plt.savefig(os.path.join(output_base_dir, 'figures/feature_importance.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(output_base_dir, 'figures/feature_importance.pdf'), bbox_inches='tight')
plt.close()

# ---------- 7) Patient-Level Data Splitting ----------
print("\n" + "="*80)
print("7. PATIENT-LEVEL DATA SPLITTING")
print("="*80)

# Get unique patients
unique_patients = df['patient_id'].unique()
patient_status = df.groupby('patient_id')['status'].first()

# Stratified split by patient
train_patients, temp_patients = train_test_split(
    unique_patients, test_size=0.3, random_state=42,
    stratify=patient_status
)
val_patients, test_patients = train_test_split(
    temp_patients, test_size=0.5, random_state=42,
    stratify=patient_status[patient_status.index.isin(temp_patients)]
)

# Create masks
train_mask = df['patient_id'].isin(train_patients)
val_mask = df['patient_id'].isin(val_patients)
test_mask = df['patient_id'].isin(test_patients)

# Apply splits
X_train = X_scaled[train_mask]
X_val = X_scaled[val_mask]
X_test = X_scaled[test_mask]
y_train = y[train_mask]
y_val = y[val_mask]
y_test = y[test_mask]

# Use only selected features
X_train_sel = X_train[selected_features]
X_val_sel = X_val[selected_features]
X_test_sel = X_test[selected_features]

print(f"Training set: {X_train_sel.shape[0]} samples ({len(train_patients)} patients)")
print(f"Validation set: {X_val_sel.shape[0]} samples ({len(val_patients)} patients)")
print(f"Test set: {X_test_sel.shape[0]} samples ({len(test_patients)} patients)")

# Save split information
split_info = pd.DataFrame({
    'patient_id': df['patient_id'],
    'split': ['train' if m else 'val' if v else 'test'
              for m, v, t in zip(train_mask, val_mask, test_mask)]
})
split_info.to_csv(os.path.join(output_base_dir, 'tables/data_splits.csv'), index=False)

# ---------- 8) Handle Class Imbalance with SMOTE ----------
print("\n" + "="*80)
print("8. HANDLING CLASS IMBALANCE")
print("="*80)

print(f"Before SMOTE - Class distribution: {pd.Series(y_train).value_counts().to_dict()}")

smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train_sel, y_train)

print(f"After SMOTE - Class distribution: {pd.Series(y_train_smote).value_counts().to_dict()}")

# ---------- 9) Model Training and Evaluation ----------
print("\n" + "="*80)
print("9. MODEL TRAINING AND EVALUATION")
print("="*80)

# Define models
models = {
    'Logistic Regression': LogisticRegression(random_state=42, max_iter=1000),
    'Random Forest': RandomForestClassifier(n_estimators=200, random_state=42, n_jobs=-1),
    'Support Vector Machine': SVC(probability=True, random_state=42),
    'XGBoost': XGBClassifier(random_state=42, eval_metric='logloss', n_estimators=200),
    'LightGBM': lgb.LGBMClassifier(random_state=42, n_estimators=200),
    'Gradient Boosting': GradientBoostingClassifier(random_state=42, n_estimators=200)
}

# Train and evaluate models
model_results = {}
trained_models = {}

for name, model in models.items():
    print(f"\nTraining {name}...")

    # Train model
    model.fit(X_train_smote, y_train_smote)
    trained_models[name] = model

    # Predictions
    y_pred = model.predict(X_test_sel)
    y_pred_proba = model.predict_proba(X_test_sel)[:, 1]

    # Calculate metrics
    accuracy = accuracy_score(y_test, y_pred)
    precision = precision_score(y_test, y_pred, zero_division=0)
    recall = recall_score(y_test, y_pred, zero_division=0)
    f1 = f1_score(y_test, y_pred, zero_division=0)
    roc_auc = roc_auc_score(y_test, y_pred_proba)

    # Cross-validation scores
    cv_scores = cross_val_score(model, X_train_smote, y_train_smote,
                               cv=5, scoring='accuracy')
    cv_mean = cv_scores.mean()
    cv_std = cv_scores.std()

    model_results[name] = {
        'model': model,
        'accuracy': accuracy,
        'precision': precision,
        'recall': recall,
        'f1_score': f1,
        'roc_auc': roc_auc,
        'cv_mean': cv_mean,
        'cv_std': cv_std,
        'y_pred': y_pred,
        'y_pred_proba': y_pred_proba
    }

    print(f"  Accuracy: {accuracy:.4f}, F1: {f1:.4f}, AUC: {roc_auc:.4f}")
    print(f"  CV Accuracy: {cv_mean:.4f} ± {cv_std:.4f}")

# ---------- 10) Comprehensive Model Visualization ----------
print("\n" + "="*80)
print("10. COMPREHENSIVE MODEL VISUALIZATION")
print("="*80)

# 10.1 Performance Comparison Bar Chart
metrics = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
metric_names = ['Accuracy', 'Precision', 'Recall', 'F1-Score', 'ROC-AUC']

plt.figure(figsize=(15, 10))
for i, (metric, metric_name) in enumerate(zip(metrics, metric_names)):
    plt.subplot(2, 3, i+1)
    values = [model_results[model][metric] for model in models.keys()]
    bars = plt.barh(list(models.keys()), values, color=plt.cm.Set3(np.linspace(0, 1, len(models))))
    plt.xlabel(metric_name)
    plt.xlim(0, 1)
    # Add value labels on bars
    for bar, value in zip(bars, values):
        plt.text(bar.get_width() + 0.01, bar.get_y() + bar.get_height()/2,
                f'{value:.3f}', ha='left', va='center', fontsize=8)
    plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig(os.path.join(output_base_dir, 'figures/model_performance_comparison.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(output_base_dir, 'figures/model_performance_comparison.pdf'), bbox_inches='tight')
plt.close()

# 10.2 Confusion Matrices
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

for i, (name, results) in enumerate(model_results.items()):
    cm = confusion_matrix(y_test, results['y_pred'])
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=axes[i],
                xticklabels=['Healthy', 'Parkinson\'s'],
                yticklabels=['Healthy', 'Parkinson\'s'])
    axes[i].set_title(f'{name}\nAccuracy: {results["accuracy"]:.3f}', fontsize=12)
    axes[i].set_xlabel('Predicted')
    axes[i].set_ylabel('Actual')

# Remove empty subplot if needed
if len(models) < 6:
    for j in range(len(models), 6):
        axes[j].axis('off')

plt.tight_layout()
plt.savefig(os.path.join(output_base_dir, 'figures/confusion_matrices.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(output_base_dir, 'figures/confusion_matrices.pdf'), bbox_inches='tight')
plt.close()

# 10.3 ROC Curves
plt.figure(figsize=(10, 8))
for name, results in model_results.items():
    fpr, tpr, _ = roc_curve(y_test, results['y_pred_proba'])
    roc_auc = auc(fpr, tpr)
    plt.plot(fpr, tpr, lw=2, label=f'{name} (AUC = {roc_auc:.3f})')

plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--', alpha=0.5)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('Receiver Operating Characteristic (ROC) Curves')
plt.legend(loc="lower right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(output_base_dir, 'figures/roc_curves.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(output_base_dir, 'figures/roc_curves.pdf'), bbox_inches='tight')
plt.close()

# 10.4 Precision-Recall Curves
plt.figure(figsize=(10, 8))
for name, results in model_results.items():
    precision_vals, recall_vals, _ = precision_recall_curve(y_test, results['y_pred_proba'])
    plt.plot(recall_vals, precision_vals, lw=2, label=f'{name}')

plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Precision-Recall Curves')
plt.legend(loc="upper right")
plt.grid(True, alpha=0.3)
plt.tight_layout()
plt.savefig(os.path.join(output_base_dir, 'figures/precision_recall_curves.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(output_base_dir, 'figures/precision_recall_curves.pdf'), bbox_inches='tight')
plt.close()

# ---------- 11) SHAP Explainable AI ----------
print("\n" + "="*80)
print("11. SHAP EXPLAINABLE AI ANALYSIS")
print("="*80)

# Use the best model for SHAP analysis
best_model_name = max(model_results.keys(), key=lambda x: model_results[x]['roc_auc'])
best_model = model_results[best_model_name]['model']
print(f"Using {best_model_name} for SHAP analysis (AUC: {model_results[best_model_name]['roc_auc']:.4f})")

try:
    # Create SHAP explainer
    explainer = shap.TreeExplainer(best_model)

    # Calculate SHAP values
    shap_values = explainer.shap_values(X_test_sel)

    # Handle binary classification output
    if isinstance(shap_values, list) and len(shap_values) == 2:
        shap_values = shap_values[1]  # Use positive class

    # 11.1 SHAP Summary Plot
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_test_sel, feature_names=selected_features, show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(output_base_dir, 'figures/shap_summary.png'), bbox_inches='tight', dpi=300)
    plt.savefig(os.path.join(output_base_dir, 'figures/shap_summary.pdf'), bbox_inches='tight')
    plt.close()

    # 11.2 SHAP Bar Plot (Mean Absolute SHAP Values)
    plt.figure(figsize=(10, 8))
    shap.summary_plot(shap_values, X_test_sel, feature_names=selected_features,
                     plot_type="bar", show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(output_base_dir, 'figures/shap_bar_plot.png'), bbox_inches='tight', dpi=300)
    plt.savefig(os.path.join(output_base_dir, 'figures/shap_bar_plot.pdf'), bbox_inches='tight')
    plt.close()

    # 11.3 SHAP Force Plot for a specific instance
    plt.figure(figsize=(12, 4))
    # Check if expected_value is a list (multi-output) and use the positive class expected value
    expected_value = explainer.expected_value[1] if isinstance(explainer.expected_value, list) else explainer.expected_value
    # Ensure the correct arguments are passed to force_plot
    shap.force_plot(expected_value, shap_values[0], X_test_sel.iloc[0], feature_names=selected_features,
                   matplotlib=True, show=False)
    plt.tight_layout()
    plt.savefig(os.path.join(output_base_dir, 'figures/shap_force_plot.png'), bbox_inches='tight', dpi=300)
    plt.savefig(os.path.join(output_base_dir, 'figures/shap_force_plot.pdf'), bbox_inches='tight')
    plt.close()


    print("SHAP analysis completed successfully")

except Exception as e:
    print(f"SHAP analysis failed: {e}")
    print("Proceeding without SHAP visualizations")

# ---------- 12) Create Combined Results Dashboard ----------
print("\n" + "="*80)
print("12. CREATING COMBINED RESULTS DASHBOARD")
print("="*80)

# 12.1 Create comprehensive results table
results_summary = []
for name, results in model_results.items():
    results_summary.append({
        'Model': name,
        'Accuracy': f"{results['accuracy']:.4f}",
        'Precision': f"{results['precision']:.4f}",
        'Recall': f"{results['recall']:.4f}",
        'F1-Score': f"{results['f1_score']:.4f}",
        'ROC-AUC': f"{results['roc_auc']:.4f}",
        'CV Accuracy (Mean)': f"{results['cv_mean']:.4f}",
        'CV Accuracy (Std)': f"{results['cv_std']:.4f}"
    })

results_df = pd.DataFrame(results_summary)
results_df.to_csv(os.path.join(output_base_dir, 'tables/comprehensive_results.csv'), index=False)
results_df.to_excel(os.path.join(output_base_dir, 'tables/comprehensive_results.xlsx'), index=False)

# 12.2 Create a summary visualization
fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, figsize=(16, 12))

# Model performance comparison
metrics_plot = ['accuracy', 'precision', 'recall', 'f1_score', 'roc_auc']
x_pos = np.arange(len(models))
width = 0.15

for i, metric in enumerate(metrics_plot):
    values = [model_results[model][metric] for model in models.keys()]
    ax1.bar(x_pos + i*width, values, width, label=metric.replace('_', ' ').title())

ax1.set_xlabel('Models')
ax1.set_ylabel('Score')
ax1.set_title('Model Performance Comparison')
ax1.set_xticks(x_pos + width*2)
ax1.set_xticklabels(list(models.keys()), rotation=45)
ax1.legend()
ax1.grid(True, alpha=0.3)

# ROC Curves
for name, results in model_results.items():
    fpr, tpr, _ = roc_curve(y_test, results['y_pred_proba'])
    roc_auc = auc(fpr, tpr)
    ax2.plot(fpr, tpr, label=f'{name} (AUC = {roc_auc:.3f})')

ax2.plot([0, 1], [0, 1], 'k--', alpha=0.5)
ax2.set_xlabel('False Positive Rate')
ax2.set_ylabel('True Positive Rate')
ax2.set_title('ROC Curves')
ax2.legend()
ax2.grid(True, alpha=0.3)

# Feature Importance
ax3.barh(range(len(top_features)), top_features['Importance'])
ax3.set_yticks(range(len(top_features)))
ax3.set_yticklabels(top_features['Feature'])
ax3.set_xlabel('Importance Score')
ax3.set_title('Top Feature Importance')
ax3.grid(True, alpha=0.3)

# Class Distribution
class_counts = [dataset_info['Healthy Controls'], dataset_info['Parkinson Patients']]
ax4.pie(class_counts, labels=['Healthy', 'Parkinson\'s'], autopct='%1.1f%%',
        colors=['lightgreen', 'lightcoral'], startangle=90)
ax4.set_title('Class Distribution')

plt.tight_layout()
plt.savefig(os.path.join(output_base_dir, 'figures/combined_results_dashboard.png'), bbox_inches='tight', dpi=300)
plt.savefig(os.path.join(output_base_dir, 'figures/combined_results_dashboard.pdf'), bbox_inches='tight')
plt.close()

# ---------- 13) Save Models and Preprocessors ----------
print("\n" + "="*80)
print("13. SAVING MODELS AND PREPROCESSORS")
print("="*80)

# Save all trained models
for name, model in trained_models.items():
    joblib.dump(model, os.path.join(output_base_dir, f'models/model_{name.replace(" ", "_").lower()}.pkl'))

# Save preprocessors
joblib.dump(scaler, os.path.join(output_base_dir, 'models/scaler.pkl'))
joblib.dump(imputer, os.path.join(output_base_dir, 'models/imputer.pkl'))
joblib.dump(selector, os.path.join(output_base_dir, 'models/feature_selector.pkl'))

print("All models and preprocessors saved successfully")

# ---------- 14) Generate Final Report ----------
print("\n" + "="*80)
print("14. GENERATING FINAL REPORT")
print("="*80)

# Create a comprehensive report
report_content = f"""
PARKINSON'S DISEASE DETECTION - COMPREHENSIVE ANALYSIS REPORT
================================================================================

EXPERIMENT SUMMARY:
- Dataset: Parkinsson disease.csv
- Total Samples: {dataset_info['Total Samples']}
- Features: {dataset_info['Total Features']}
- Parkinson's Patients: {dataset_info['Parkinson Patients']} ({dataset_info['Parkinson Percentage']})
- Healthy Controls: {dataset_info['Healthy Controls']} ({dataset_info['Healthy Percentage']})
- Unique Patients: {df['patient_id'].nunique()}

DATA SPLITS:
- Training Set: {X_train_sel.shape[0]} samples
- Validation Set: {X_val_sel.shape[0]} samples
- Test Set: {X_test_sel.shape[0]} samples

BEST PERFORMING MODEL: {best_model_name}
- Test Accuracy: {model_results[best_model_name]['accuracy']:.4f}
- Test F1-Score: {model_results[best_model_name]['f1_score']:.4f}
- Test ROC-AUC: {model_results[best_model_name]['roc_auc']:.4f}
- CV Accuracy: {model_results[best_model_name]['cv_mean']:.4f} ± {model_results[best_model_name]['cv_std']:.4f}

TOP 5 FEATURES:
{top_features[['Feature', 'Importance']].head().to_string(index=False)}

MODEL PERFORMANCE RANKING (by ROC-AUC):
"""

# Add model ranking
ranking = sorted(model_results.items(), key=lambda x: x[1]['roc_auc'], reverse=True)
for i, (name, results) in enumerate(ranking, 1):
    report_content += f"{i}. {name}: AUC = {results['roc_auc']:.4f}, F1 = {results['f1_score']:.4f}\n"

report_content += f"""
FILES GENERATED:

FIGURES:
- class_distribution.png/pdf - Class distribution and feature visualization
- correlation_matrix.png/pdf - Feature correlation heatmap
- dimensionality_reduction.png/pdf - PCA and t-SNE visualizations
- feature_importance.png/pdf - Feature importance ranking
- model_performance_comparison.png/pdf - Model comparison across metrics
- confusion_matrices.png/pdf - Confusion matrices for all models
- roc_curves.png/pdf - ROC curves for model comparison
- precision_recall_curves.png/pdf - Precision-Recall curves
- shap_*.png/pdf - SHAP explainability plots
- combined_results_dashboard.png/pdf - Comprehensive results summary

TABLES:
- feature_importance.csv - Complete feature importance scores
- data_splits.csv - Train/validation/test split information
- comprehensive_results.csv/xlsx - Detailed model performance metrics

MODELS:
- model_*.pkl - All trained machine learning models
- preprocessors.pkl - Scaler, imputer, and feature selector

This analysis provides a comprehensive evaluation of machine learning models for Parkinson's disease detection using voice features.
All results are suitable for conference paper submission.
"""

# Save report
with open(os.path.join(output_base_dir, 'results/comprehensive_analysis_report.txt'), 'w') as f:
    f.write(report_content)

print(report_content)

# ---------- 15) Final Summary ----------
print("\n" + "="*80)
print("EXPERIMENT COMPLETED SUCCESSFULLY!")
print("="*80)

best_model = max(model_results.items(), key=lambda x: x[1]['roc_auc'])
print(f"\nBEST PERFORMING MODEL: {best_model[0]}")
print(f"ROC-AUC: {model_results[best_model[0]]['roc_auc']:.4f}")
print(f"Accuracy: {model_results[best_model[0]]['accuracy']:.4f}")
print(f"F1-Score: {model_results[best_model[0]]['f1_score']:.4f}")

print(f"\nALL OUTPUTS SAVED IN:")
print(f"📊 {os.path.join(output_base_dir, 'figures/')} - All visualizations (PNG/PDF)")
print(f"📋 {os.path.join(output_base_dir, 'tables/')} - All data tables (CSV/XLSX)")
print(f"🤖 {os.path.join(output_base_dir, 'models/')} - Trained models and preprocessors")
print(f"📄 {os.path.join(output_base_dir, 'results/')} - Comprehensive analysis report")

print(f"\n🎯 READY FOR CONFERENCE PAPER SUBMISSION!")

ERROR: Could not find a version that satisfies the requirement boruta_shap (from versions: none)
ERROR: No matching distribution found for boruta_shap
Imports completed successfully
SHAP version: 0.48.0
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/figures
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/tables
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/models
Created directory: /content/drive/MyDrive/ML COURSE PAPER/OUTPUT FOR EVERY RUN/results

3. DATA LOADING AND EXPLORATION
Dataset shape: (195, 24)
Columns after sanitizing: ['name', 'MDVPFoHz', 'MDVPFhiHz', 'MDVPFloHz', 'MDVPJitter', 'MDVPJitterAbs', 'MDVPRAP', 'MDVPPPQ', 'JitterDDP', 'MDVPShimmer', 'MDVPShimmerdB', 'ShimmerAPQ3', 'ShimmerAPQ5', 'MDVPAPQ', 'ShimmerDDA', 'NHR', 'HNR', 'status', 'RPDE', 'DFA', 'spread1', 'spread2', 'D2', 'PPE']

Dataset Summary:
Total Samples: 195
Total Features: 22
Parkinson Patients: 147
Healthy Contr

<Figure size 3000x2400 with 0 Axes>

<Figure size 3000x2400 with 0 Axes>

<Figure size 3600x1200 with 0 Axes>